

Important note in the numerical implementation of the factorized waveform:
-  spinning component PN amplitude modes $f^{\rm S}_{lm}$ (defined in [this cell](#fspin)), there are terms of $\mathcal{O}(\frac{\chi_A}{\delta})$ that diverge in the equal mass limit, i.e, $\delta \rightarrow 0$. However, for all odd values of $m$, the Newtonian multipole coefficient $c_{l + \epsilon}$ behaves as $c_{l + \epsilon} \propto \delta[1 + \mathcal{O}(\delta)]$ and vanishes in the equal mass $\delta \rightarrow 0$ limit. Therefore, the product $c_{l+\epsilon}f^{\rm S}_{lm}$ converges in the equal mass limit. In order to ensure that we obtain the mathematically correct flux in the equal mass limit, we choose our tolerance as $10^{-14}$ for the limit. That is, for $\delta > 10^{-14}$, we calculate $c_{l+\epsilon}$ and $f^{\rm S}_{lm}$ normally. For $\delta < 10^{-14}$, we calculate $\lim_{\delta \rightarrow 0}\left(\frac{c_{l + \epsilon}}{\delta}\right)$ and $\lim_{\delta \rightarrow 0}\left(\delta f^{\rm NS}_{lm}\right)$ instead.

- The original implementation of SEOBNR approximants rely on if-else/switch-case statements to populate waveform mode structures. In our implementation, we list each $l,m$ mode explicitly to avoid if-else statements inside a for loop for smoother computation. Additionally, the prescription given in the [NRPy+ Piecewise Expressions Tutorial](https://nbviewer.org/github/zachetienne/nrpytutorial/blob/master/Tutorial-Min_Max_and_Piecewise_Expressions.ipynb) will be used to generate conditional expressions to avoid checking for the equal mass limit discussed above.

-  As per the discussion above Equation 21 in [DCCT2300060](https://dcc.ligo.org/public/0186/T2300060/002/SEOBNRv5HM.pdf), the SEOBNRv5HM waveform only uses the (2,2), (2,1), (3,3), (3,2), (4,4), (4,3), and (5,5) modes to build the waveform. However, the flux is computed by summing up the $l \leq 8$ mode amplitudes. In this notebook, we list all the waveform modes for completeness. The resulting code, however, is generated by building only relevant modes as sympy expressions. This allows only for relevant parts of the notebook to enter the final SEOBNRv5_NRPy_opt code, making our implementation less bulky. (Future approximants may add or build on additional modes. thus, this notebook facilitates extensibility to future approximants!)
\end{itemize}

In [ ]:
import sys#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Raditation"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

<a id='hlm'></a>

# Step : The Factorized Waveform $h^{\rm F}_{lm}$ \[Back to [top](#toc)\]
$$\label{hlm}$$

The factorized inspiral waveform modes are expressed in Equation 25 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
h^{\rm F}_{lm} = h^{\rm N}_{lm}\hat{S}_{\rm eff}T_{lm}f_{lm}e^{i\delta_{lm}}
$$

Where, $h^{\rm N}_{lm}$ is the leading (Newtonian) order waveform defined in [this cell](#hnewtonian), $\hat{S}_{\rm eff}$ is the effective source term defined in [this cell](#source), $T_{lm}$ is the contribution to the waveform obtained by resumming tail contributions defined in [this cell](#tail), $f_{lm}$ is the PN contribution to the amplitude defined in [this cell](#pn), and $\delta_{lm}$ is the PN contribution to the phase.

We note, first that the phase information is only relevant for (2,2), (2,1), (3,3), (3,2), (4,4), (4,3), and (5,5) modes for the waveform. The remaining modes are only used to calculate the flux for which we only require the amplitude component i.e $|h^{\rm F}_{lm}|$.

In [ ]:
%%writefile $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
h22 = hN22*source_even*T22*f22*(sp.cos(delta22) + sp.I*sp.sin(delta22))
h22amp = hN22amp*source_even*T22amp*f22amp
h21 = hN21*source_odd*T21*f21*(sp.cos(delta21) + sp.I*sp.sin(delta21))
h21amp = hN21amp*source_odd*T21amp*f21amp
h33 = hN33*source_even*T33*f33*(sp.cos(delta33) + sp.I*sp.sin(delta33))
h33amp = hN33amp*source_even*T33amp*f33amp
h32 = hN32*source_odd*T32*f32*(sp.cos(delta32) + sp.I*sp.sin(delta32))
h32amp = hN32amp*source_odd*T32amp*f32amp
h31amp = hN31amp*source_even*T31amp*f31amp
h44 = hN44*source_even*T44*f44*(sp.cos(delta44) + sp.I*sp.sin(delta44))
h44amp = hN44amp*source_even*T44amp*f44amp
h43 = hN43*source_odd*T43*f43*(sp.cos(delta43) + sp.I*sp.sin(delta43))
h43amp = hN43amp*source_odd*T43amp*f43amp
h42amp = hN42amp*source_even*T42amp*f42amp
h41amp = hN41amp*source_odd*T41amp*f41amp
h55 = hN55*source_even*T55*f55*(sp.cos(delta55) + sp.I*sp.sin(delta55))
h55amp = hN55amp*source_even*T55amp*f55amp
h54amp = hN54amp*source_odd*T54amp*f54amp
h53amp = hN53amp*source_even*T53amp*f53amp
h52amp = hN52amp*source_odd*T52amp*f52amp
h51amp = hN51amp*source_even*T51amp*f51amp
h66amp = hN66amp*source_even*T66amp*f66amp
h65amp = hN65amp*source_odd*T65amp*f65amp
h64amp = hN64amp*source_even*T64amp*f64amp
h63amp = hN63amp*source_odd*T63amp*f63amp
h62amp = hN62amp*source_even*T62amp*f62amp
h61amp = hN61amp*source_odd*T61amp*f61amp
h77amp = hN77amp*source_even*T77amp*f77amp
h76amp = hN76amp*source_odd*T76amp*f76amp
h75amp = hN75amp*source_even*T75amp*f75amp
h74amp = hN74amp*source_odd*T74amp*f74amp
h73amp = hN73amp*source_even*T73amp*f73amp
h72amp = hN72amp*source_odd*T72amp*f72amp
h71amp = hN71amp*source_even*T71amp*f71amp
h88amp = hN88amp*source_even*T88amp*f88amp
h87amp = hN87amp*source_odd*T87amp*f87amp
h86amp = hN86amp*source_even*T86amp*f86amp
h85amp = hN85amp*source_odd*T85amp*f85amp
h84amp = hN84amp*source_even*T84amp*f84amp
h83amp = hN83amp*source_odd*T83amp*f83amp
h82amp = hN82amp*source_even*T82amp*f82amp
h81amp = hN81amp*source_odd*T81amp*f81amp

<a id='hnewtonian'></a>

# Step : The Newtonian Waveform $h^{\rm N}_{lm}$ \[Back to [top](#toc)\]
$$\label{hnewtonian}$$

The Newtonian component of the inspiral waveform modes are expressed in Equation 26 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
h^{\rm N}_{lm} = \eta n_{lm}c_{l + \epsilon_{lm}}v_{\phi}^{l + \epsilon_{lm}}Y_{l - \epsilon_{lm},-m}\left(\frac{\pi}{2},\phi\right)
$$

Where, the symmetric mass ratio $\eta$ is defined in [this cell], the $\eta$-independent coefficient $n_{lm}$ is defined in [this cell](#n), the $\eta$-dependent coefficient $c_{l + \epsilon_{lm}}$ is defined in [this cell](#c), the PN circular velocity $v_{\phi}$ is defined in [this cell](#v), and $Y$ represents the spherical harmonic function listed in [this cell](#y). The parity $\epsilon_{lm}$ of the mode is given by

$$
\epsilon_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        0, & l + m \textrm{ even}\\
        1, & l + m \textrm{ odd}
    \end{array}
$$

$\phi$ is the orbital phase, given as an input.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
hN22 = eta*N22*c2*vphi**2*Y22
hN22amp = eta*N22amp*c2*vphi**2*Y22amp
hN21 = eta*N21*c3*vphi**3*Y11
hN21amp = eta*N21amp*c3*vphi**3*Y11amp
hN33 = eta*N33*c3*vphi**3*Y33
hN33amp = eta*N33amp*c3*vphi**3*Y33amp
hN32 = eta*N32*c4*vphi**4*Y22
hN32amp = eta*N32amp*c4*vphi**4*Y22amp
hN31amp = eta*N31amp*c3*vphi**3*Y31amp
hN44 = eta*N44*c4*vphi**4*Y44
hN44amp = eta*N44amp*c4*vphi**4*Y44amp
hN43 = eta*N43*c5*vphi**5*Y33
hN43amp = eta*N43amp*c5*vphi**5*Y33amp
hN42amp = eta*N42amp*c4*vphi**4*Y42amp
hN41amp = eta*N41amp*c5*vphi**5*Y31amp
hN55 = eta*N55*c5*vphi**5*Y55
hN55amp = eta*N55amp*c5*vphi**5*Y55amp
hN54amp = eta*N54amp*c6*vphi**6*Y44amp
hN53amp = eta*N53amp*c5*vphi**5*Y53amp
hN52amp = eta*N52amp*c6*vphi**6*Y42amp
hN51amp = eta*N51amp*c5*vphi**5*Y51amp
hN66amp = eta*N66amp*c6*vphi**6*Y66amp
hN65amp = eta*N65amp*c7*vphi**7*Y55amp
hN64amp = eta*N64amp*c6*vphi**6*Y64amp
hN63amp = eta*N63amp*c7*vphi**7*Y53amp
hN62amp = eta*N62amp*c6*vphi**6*Y62amp
hN61amp = eta*N61amp*c7*vphi**7*Y51amp
hN77amp = eta*N77amp*c7*vphi**7*Y77amp
hN76amp = eta*N76amp*c8*vphi**8*Y66amp
hN75amp = eta*N75amp*c7*vphi**7*Y75amp
hN74amp = eta*N74amp*c8*vphi**8*Y64amp
hN73amp = eta*N73amp*c7*vphi**7*Y73amp
hN72amp = eta*N72amp*c8*vphi**8*Y62amp
hN71amp = eta*N71amp*c7*vphi**7*Y71amp
hN88amp = eta*N88amp*c8*vphi**8*Y88amp
hN87amp = eta*N87amp*c9*vphi**9*Y77amp
hN86amp = eta*N86amp*c8*vphi**8*Y86amp
hN85amp = eta*N85amp*c9*vphi**9*Y75amp
hN84amp = eta*N84amp*c8*vphi**8*Y84amp
hN83amp = eta*N83amp*c9*vphi**9*Y73amp
hN82amp = eta*N82amp*c8*vphi**8*Y82amp
hN81amp = eta*N81amp*c9*vphi**9*Y71amp

<a id='source'></a>

# Step : The Source Term $\hat{S}_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{source}$$

The tail component of the inspiral waveform modes are expressed in Equation 32 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
\hat{S}_{\rm eff} = 
\bigg\lbrace
    \begin{array}{lr}
        H_{\rm eff} & l + m \textrm{ even},\\
        v_{\Omega}p_{\phi} & l + m \textrm{ odd},
    \end{array}
$$

Where, $H_{\rm eff}$,the effective Hamiltonian, is defined in [this cell](#heff), and v_{\Omega} is defined in [this cell](#v). $p_{\phi}$ is the azimuthal momentum which is given as an input.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
source_even = Heff
source_odd = vomega*pphi

<a id='tail'></a>

# Step : The Tail Term $T_{lm}$ \[Back to [top](#toc)\]
$$\label{tail}$$

The tail component of the inspiral waveform modes are expressed in Equation 33 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
T_{lm} = -\frac{\Gamma (l + 1 - 2i\hat{k})}{\Gamma(l + 1)}e^{\pi \hat{k}}e^{2i\hat{k}\ln(2m\Omega r_0)}
$$

Where, $\Gamma$ is the gamma function implemented in sympy and defined as in [this Wikipedia article](https://en.wikipedia.org/wiki/Gamma_function). wavenumber $\hat{k}$ is defined in [this cell](#khat), the PN frequency $\Omega$ is defined in [this cell](#omegapn), and the constant $r_0$ is defined in [this cell](#r0).

The above expression is optimized mathematically before coding to exploit the properties of the gamma function. In this case, we note the simplification:

$$
\Gamma(l + 1) = l! \quad \forall l \in \mathbb{N}.
$$

which does not require using a generalized gamma function call.

Additionally, the full gamma function is not required when calculating the amplitude. Particularly, cf the properties section of the same [Wikipedia article](https://en.wikipedia.org/wiki/Gamma_function#Properties) to find the following simplification

$$
|\Gamma(1 + l + bi)|^2 = \frac{\pi b}{\sinh \pi b} \prod_{j = 1}^{l} (j^2 + b^2) = \frac{2\pi b}{e^{\pi b}- e^{-\pi b}} \prod_{j = 1}^{l} (j^2 + b^2)
$$

Where, in the last step, $\sinh$ is defined explicitly as sympy's cse does not exploit the definition of the $\sinh$ function.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
T22 = -sp.gamma(2 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*2*omega*r0)))/sp.factorial(2)
T22amp = -gamma_amp_2*sp.exp(khatpi)/sp.factorial(2)
T21 = -sp.gamma(2 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*1*omega*r0)))/sp.factorial(2)
T21amp = -gamma_amp_2*sp.exp(khatpi)/sp.factorial(2)
T33 = -sp.gamma(3 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*3*omega*r0)))/sp.factorial(3)
T33amp = -gamma_amp_3*sp.exp(khatpi)/sp.factorial(3)
T32 = -sp.gamma(3 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*2*omega*r0)))/sp.factorial(3)
T32amp = -gamma_amp_3*sp.exp(khatpi)/sp.factorial(3)
T31amp = -sp.gamma(3 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*1*omega*r0)))/sp.factorial(3)
T44 = -sp.gamma(4 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*4*omega*r0)))/sp.factorial(4)
T44amp = -gamma_amp_4*sp.exp(khatpi)/sp.factorial(4)
T43 = -sp.gamma(4 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*3*omega*r0)))/sp.factorial(4)
T43amp = -gamma_amp_4*sp.exp(khatpi)/sp.factorial(4)
T42amp = -gamma_amp_4*sp.exp(khatpi)/sp.factorial(4)
T41amp = -gamma_amp_4*sp.exp(khatpi)/sp.factorial(4)
T55 = -sp.gamma(5 + 1 - 2*sp.I*khat)*sp.exp(khatpi)*(sp.exp(2*sp.I*khat*sp.log(2*5*omega*r0)))/sp.factorial(5)
T55amp = -gamma_amp_5*sp.exp(khatpi)/sp.factorial(5)
T54amp = -gamma_amp_5*sp.exp(khatpi)/sp.factorial(5)
T53amp = -gamma_amp_5*sp.exp(khatpi)/sp.factorial(5)
T52amp = -gamma_amp_5*sp.exp(khatpi)/sp.factorial(5)
T51amp = -gamma_amp_5*sp.exp(khatpi)/sp.factorial(5)
T66amp = -gamma_amp_6*sp.exp(khatpi)/sp.factorial(6)
T65amp = -gamma_amp_6*sp.exp(khatpi)/sp.factorial(6)
T64amp = -gamma_amp_6*sp.exp(khatpi)/sp.factorial(6)
T63amp = -gamma_amp_6*sp.exp(khatpi)/sp.factorial(6)
T62amp = -gamma_amp_6*sp.exp(khatpi)/sp.factorial(6)
T61amp = -gamma_amp_6*sp.exp(khatpi)/sp.factorial(6)
T77amp = -gamma_amp_7*sp.exp(khatpi)/sp.factorial(7)
T76amp = -gamma_amp_7*sp.exp(khatpi)/sp.factorial(7)
T75amp = -gamma_amp_7*sp.exp(khatpi)/sp.factorial(7)
T74amp = -gamma_amp_7*sp.exp(khatpi)/sp.factorial(7)
T73amp = -gamma_amp_7*sp.exp(khatpi)/sp.factorial(7)
T72amp = -gamma_amp_7*sp.exp(khatpi)/sp.factorial(7)
T71amp = -gamma_amp_7*sp.exp(khatpi)/sp.factorial(7)
T88amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
T87amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
T86amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
T85amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
T84amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
T83amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
T82amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
T81amp = -gamma_amp_8*sp.exp(khatpi)/sp.factorial(8)
gamma_amp_8 = sp.sqrt(Tlmprefac*j2plusb2_l8)
gamma_amp_7 = sp.sqrt(Tlmprefac*j2plusb2_l7)
gamma_amp_6 = sp.sqrt(Tlmprefac*j2plusb2_l6)
gamma_amp_5 = sp.sqrt(Tlmprefac*j2plusb2_l5)
gamma_amp_4 = sp.sqrt(Tlmprefac*j2plusb2_l4)
gamma_amp_3 = sp.sqrt(Tlmprefac*j2plusb2_l3)
gamma_amp_2 = sp.sqrt(Tlmprefac*j2plusb2_l2)
j2plusb2_l8 = j2plusb2_l7*(8**2 + khat2**2)
j2plusb2_l7 = j2plusb2_l6*(7**2 + khat2**2)
j2plusb2_l6 = j2plusb2_l5*(6**2 + khat2**2)
j2plusb2_l5 = j2plusb2_l4*(5**2 + khat2**2)
j2plusb2_l4 = j2plusb2_l3*(4**2 + khat2**2)
j2plusb2_l3 = j2plusb2_l2*(3**2 + khat2**2)
j2plusb2_l2 = (1**2 + khat**2)*(2**2 + khat2**2)
Tlmprefac = 2*khatpi/(sp.exp(khatpi) - sp.exp(-khatpi))
khat2 = 2*khat
khatpi = khat*sp.pi

# Step : The Newtonian Waveform Components

<a id='n'></a>

# Step : The coefficient $n_{lm}$ \[Back to [top](#toc)\]
$$\label{n}$$

The $\eta$-independent coefficient of the Newtonian waveform modes is expressed in Equation 28 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
{n}_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        \frac{8\pi(im)^l}{(2l + 1)!!}\sqrt{\frac{(l+1)(l+2)}{(l)(l-1)}} & l + m \textrm{ even},\\
        \frac{-16\pi(im)^l}{(2l + 1)!!}\sqrt{\frac{(2l+1)(l+2)(l^2 - m^2)}{(2l-1)(l+1)(l)(l-1)}} & l + m \textrm{ odd},
    \end{array}
$$

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
N22 = (sp.I)**2 * N22amp
N22amp = (8*sp.pi*(2)**2)*sp.sqrt(((2 + 1)*(2+2))/(2*(2- 1)))/sp.factorial2(2*2+1)
N21 = -(sp.I)**2 * N21amp
N21amp = (16*sp.pi*(1)**2)*sp.sqrt(((2**2 - 1**2)*(2*2 + 1)*(2+2))/((2*2-1)*(2+1)*2*(2- 1)))/sp.factorial2(2*2+1)
N33 = (sp.I)**3 * N33amp
N33amp = (8*sp.pi*(3)**3)*sp.sqrt(((3 + 1)*(3+2))/(3*(3- 1)))/sp.factorial2(2*3+1)
N32 = -(sp.I)**3 * N32amp
N32amp = (16*sp.pi*(2)**3)*sp.sqrt(((3**2 - 2**2)*(2*3 + 1)*(3+2))/((2*3-1)*(3+1)*3*(3- 1)))/sp.factorial2(2*3+1)
N31amp = (8*sp.pi*(1)**3)*sp.sqrt(((3 + 1)*(3+2))/(3*(3- 1)))/sp.factorial2(2*3+1)
N44 = (sp.I)**4 * N44amp
N44amp = (8*sp.pi*(4)**4)*sp.sqrt(((4 + 1)*(4+2))/(4*(4- 1)))/sp.factorial2(2*4+1)
N43 = -(sp.I)**4 * N43amp
N43amp = (16*sp.pi*(3)**4)*sp.sqrt(((4**2 - 3**2)*(2*4 + 1)*(4+2))/((2*4-1)*(4+1)*4*(4- 1)))/sp.factorial2(2*4+1)
N42amp = (8*sp.pi*(2)**4)*sp.sqrt(((4 + 1)*(4+2))/(4*(4- 1)))/sp.factorial2(2*4+1)
N41amp = (16*sp.pi*(1)**4)*sp.sqrt(((4**2 - 1**2)*(2*4 + 1)*(4+2))/((2*4-1)*(4+1)*4*(4- 1)))/sp.factorial2(2*4+1)
N55 = (sp.I)**5 * N55amp
N55amp = (8*sp.pi*(5)**5)*sp.sqrt(((5 + 1)*(5+2))/(5*(5- 1)))/sp.factorial2(2*5+1)
N54amp = (16*sp.pi*(4)**5)*sp.sqrt(((5**2 - 4**2)*(2*5 + 1)*(5+2))/((2*5-1)*(5+1)*5*(5- 1)))/sp.factorial2(2*5+1)
N53amp = (8*sp.pi*(3)**5)*sp.sqrt(((5 + 1)*(5+2))/(5*(5- 1)))/sp.factorial2(2*5+1)
N52amp = (16*sp.pi*(2)**5)*sp.sqrt(((5**2 - 2**2)*(2*5 + 1)*(5+2))/((2*5-1)*(5+1)*5*(5- 1)))/sp.factorial2(2*5+1)
N51amp = (8*sp.pi*(1)**5)*sp.sqrt(((5 + 1)*(5+2))/(5*(5- 1)))/sp.factorial2(2*5+1)
N66amp = (8*sp.pi*(6)**6)*sp.sqrt(((6 + 1)*(6+2))/(6*(6- 1)))/sp.factorial2(2*6+1)
N65amp = (16*sp.pi*(5)**6)*sp.sqrt(((6**2 - 5**2)*(2*6 + 1)*(6+2))/((2*6-1)*(6+1)*6*(6- 1)))/sp.factorial2(2*6+1)
N64amp = (8*sp.pi*(4)**6)*sp.sqrt(((6 + 1)*(6+2))/(6*(6- 1)))/sp.factorial2(2*6+1)
N63amp = (16*sp.pi*(3)**6)*sp.sqrt(((6**2 - 3**2)*(2*6 + 1)*(6+2))/((2*6-1)*(6+1)*6*(6- 1)))/sp.factorial2(2*6+1)
N62amp = (8*sp.pi*(2)**6)*sp.sqrt(((6 + 1)*(6+2))/(6*(6- 1)))/sp.factorial2(2*6+1)
N61amp = (16*sp.pi*(1)**6)*sp.sqrt(((6**2 - 1**2)*(2*6 + 1)*(6+2))/((2*6-1)*(6+1)*6*(6- 1)))/sp.factorial2(2*6+1)
N77amp = (8*sp.pi*(7)**7)*sp.sqrt(((7 + 1)*(7+2))/(7*(7- 1)))/sp.factorial2(2*7+1)
N76amp = (16*sp.pi*(6)**7)*sp.sqrt(((7**2 - 6**2)*(2*7 + 1)*(7+2))/((2*7-1)*(7+1)*7*(7- 1)))/sp.factorial2(2*7+1)
N75amp = (8*sp.pi*(5)**7)*sp.sqrt(((7 + 1)*(7+2))/(7*(7- 1)))/sp.factorial2(2*7+1)
N74amp = (16*sp.pi*(4)**7)*sp.sqrt(((7**2 - 4**2)*(2*7 + 1)*(7+2))/((2*7-1)*(7+1)*7*(7- 1)))/sp.factorial2(2*7+1)
N73amp = (8*sp.pi*(3)**7)*sp.sqrt(((7 + 1)*(7+2))/(7*(7- 1)))/sp.factorial2(2*7+1)
N72amp = (16*sp.pi*(2)**7)*sp.sqrt(((7**2 - 2**2)*(2*7 + 1)*(7+2))/((2*7-1)*(7+1)*7*(7- 1)))/sp.factorial2(2*7+1)
N71amp = (8*sp.pi*(1)**7)*sp.sqrt(((7 + 1)*(7+2))/(7*(7- 1)))/sp.factorial2(2*7+1)
N88amp = (8*sp.pi*(8)**8)*sp.sqrt(((8 + 1)*(8+2))/(8*(8- 1)))/sp.factorial2(2*l+1)
N87amp = (16*sp.pi*(7)**8)*sp.sqrt(((8**2 - 7**2)*(2*8 + 1)*(8+2))/((2*8-1)*(8+1)*8*(8- 1)))/sp.factorial2(2*8+1)
N86amp = (8*sp.pi*(6)**8)*sp.sqrt(((8 + 1)*(8+2))/(8*(8- 1)))/sp.factorial2(2*8+1)
N85amp = (16*sp.pi*(5)**8)*sp.sqrt(((8**2 - 5**2)*(2*8 + 1)*(8+2))/((2*8-1)*(8+1)*8*(8- 1)))/sp.factorial2(2*8+1)
N84amp = (8*sp.pi*(4)**8)*sp.sqrt(((8 + 1)*(8+2))/(8*(8- 1)))/sp.factorial2(2*8+1)
N83amp = (16*sp.pi*(3)**8)*sp.sqrt(((8**2 - 3**2)*(2*8 + 1)*(8+2))/((2*8-1)*(8+1)*8*(8- 1)))/sp.factorial2(2*8+1)
N82amp = (8*sp.pi*(2)**8)*sp.sqrt(((8 + 1)*(8+2))/(8*(8- 1)))/sp.factorial2(2*8+1)
N81amp = (16*sp.pi*(1)**8)*sp.sqrt(((8**2 - 1**2)*(2*8 + 1)*(8+2))/((2*8-1)*(8+1)*8*(8- 1)))/sp.factorial2(2*8+1)

<a id='c'></a>

# Step : The coefficient $c_{l+ \epsilon_{lm}}$ \[Back to [top](#toc)\]
$$\label{c}$$

The $\eta$-dependent coefficient of the Newtonian waveform modes is expressed in Equation 29 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
{c}_{k} = \left(\frac{1 - \sqrt{1 - 4\eta}}{2}\right) + (-1)^k\left(\frac{1 + \sqrt{1 - 4\eta}}{2}\right)
$$

Since $f$ modes are only provided up to $l = 5$, we note that the values of $k = l + \epsilon_{lm}$ where the equal mass limit must be evaluated are given in the below table.

|$(l,m)$|$\epsilon_{lm}$|$k = l + \epsilon_{lm}$|$\lim_{\delta \rightarrow 0} \tfrac{c_{k}}{\delta}$|
|-------|---------------|-----------------------|----------------------------------------------------|
|$(2,1)$|$1$|$3$|$-1$|
|$(3,3)$|$0$|$3$|$-1$|
|$(3,1)$|$0$|$3$|$-1$|
|$(4,3)$|$1$|$5$|$-\tfrac{1}{2}$|
|$(4,1)$|$1$|$5$|$-\tfrac{1}{2}$|
|$(5,5)$|$0$|$5$|$-\tfrac{1}{2}$|

Also, some simple algrebra shows that $\sqrt{1 - 4\eta} = \delta$ which can be substituted in the above equation.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
c2 = ((1 - delta)/2) + ((-1)**2)*((1 + delta)/2)
c3 = (((1 - delta)/2) + ((-1)**3)*((1 + delta)/2))*noneqcond + (-1)*eqcond
c4 = ((1 - delta)/2) + ((-1)**4)*((1 + delta)/2)
c5 = (((1 - delta)/2) + ((-1)**5)*((1 + delta)/2))*noneqcond + (-sp.Rational(1,2))*eqcond
c6 = ((1 - delta)/2) + ((-1)**6)*((1 + delta)/2)
c7 = ((1 - delta)/2) + ((-1)**7)*((1 + delta)/2)
c8 = ((1 - delta)/2) + ((-1)**8)*((1 + delta)/2)
c9 = ((1 - delta)/2) + ((-1)**9)*((1 + delta)/2)

<a id='flm'></a>

# Step : The PN amplitude component $f_{lm}$ \[Back to [top](#toc)\]
$$\label{flm}$$

The tail component of the inspiral waveform modes is expressed in Equation 34 of [PB2023](https://arxiv.org/pdf/2303.18039.pdf):

$$
f_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        (\rho_{lm})^l & m \textrm{ even}\\
        (\rho^{\rm NS}_{lm})^l + f^{\rm S}_{lm} & m \textrm{ odd}
    \end{array}     
$$

Where, $\rho_{lm}$ and its non-spinning component $\rho^{\rm NS}_{lm}$ are defined in [this cell](#rho), and the PN frequency $\Omega$ is defined in [this cell](#omegapn), and the odd-$m$ spinning component $f^{\rm S}_{lm}$ is defined in [this cell](#fspin). We also note that, as per Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf), that $f$ modes are non-zero only up to $l = 5$, where only the $m = 5$ mode is non-zero.

Finally, it is important to discuss that, in the limit $\delta \rightarrow 0,\text{ } \delta \equiv \tfrac{m_1 - m_2}{M}$, only the non-vanishing term must be applied. That is, 

$$
f_{lm} = 
\bigg\lbrace
    \begin{array}{lr}
        (\rho_{lm})^l & m \textrm{ even}\\
        (\rho^{\rm NS}_{lm})^l + f^{\rm S}_{lm} & m \textrm{ odd, and } \delta \gt 10^{-14}\\
        \lim_{\delta \rightarrow 0} (\delta f^{\rm S}_{lm}) & m \textrm{ odd, and } \delta \lt 10^{-14}
    \end{array}     
$$

NOTE: Should define $1/\delta$ to avoid divergence in the limiting case.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
f22 = (rho22)**2
f21 = ((rhoNS22)**2 + fspin21)*noneqcond + fspin21_limit*eqcond
f33 = ((rhoNS33)**3 + fspin33)*noneqcond + fspin33_limit*eqcond
f32 = (rho32)**3
f31 = ((rhoNS31)**3 + fspin31)*noneqcond + fspin31_limit*eqcond
f44 = (rho44)**4
f43 = ((rhoNS43)**4 + fspin43)*noneqcond + fspin43_limit*eqcond
f42 = (rho42)**4
f41 = ((rhoNS41)**4 + fspin41)*noneqcond + fspin41_limit*eqcond
f55 = ((rhoNS55)**5 + fspin55)*noneqcond + fspin55_limit*eqcond
f54 = (rho54)**5
f53 = ((rhoNS53)**5)*noneqcond
f52 = (rho52)**5
f51 = ((rhoNS51)**5)*noneqcond
f66 = (rho66)**6
f65 = (rhoNS65)**6
f64 = (rho64)**6
f63 = (rhoNS63)**6
f62 = (rho62)**6
f61 = (rhoNS61)**6
f77 = (rhoNS77)**7
f76 = (rho76)**7
f75 = (rhoNS75)**7
f74 = (rho74)**7
f73 = (rhoNS73)**7
f72 = (rho72)**7
f71 = (rhoNS71)**7
f88 = (rho88)**8
f87 = (rhoNS87)**8
f86 = (rho886)**8
f85 = (rhoNS85)**8
f84 = (rho84)**8
f83 = (rhoNS83)**8
f82 = (rho82)**8
f81 = (rhoNS81)**8

# Step : PN Waveform Modes

<a id='rho'></a>

# Step : $\rho_{lm}$ \[Back to [top](#toc)\]
$$\label{rho}$$

$\rho_{lm}$ is defined in Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf).

We begin with the (2,2) mode given in Equation B1a:

\begin{align*}
\rho_{22} &= 1
+ v_\Omega^2 \left(\tfrac{55}{84}\eta-\tfrac{43}{42}\right)
+ v_\Omega^3 \left[\left(\tfrac{2}{3}\eta-\tfrac{2}{3}\right) \chi _S-\tfrac{2}{3}\delta\chi _A\right]
+ v_\Omega^4 \Big[
\tfrac{19583}{42336} \eta^2-\tfrac{33025}{21168}\eta-\tfrac{20555}{10584}
+ \left(\tfrac{1}{2}-2 \eta \right) \chi _A^2+\delta  \chi _A \chi _S+\tfrac{1}{2}\chi _S^2\Big] \nonumber\\
&\quad
+ v_\Omega^5 \left[\delta  \left(-\tfrac{19}{42}\eta-\tfrac{34}{21}\right) \chi _A+\left(\tfrac{209}{126}\eta^2+\tfrac{49}{18}\eta-\tfrac{34}{21}\right) \chi _S\right] \nonumber \\
&\quad
+ v_\Omega^6 \Big[
\tfrac{10620745 \eta^3}{39118464}-\tfrac{6292061 \eta^2}{3259872}+\tfrac{41 \pi^2 \eta}{192}-\tfrac{48993925 \eta}{9779616}-\tfrac{428}{105}\text{eulerlog}(2,v_\Omega)+\tfrac{1556919113}{122245200} \nonumber \\
&\qquad 
 + \delta  \left(\tfrac{89}{126}-\tfrac{781}{252}\eta\right) \chi _A \chi _S
+\left(-\tfrac{27}{14}\eta^2-\tfrac{457}{504}\eta+\tfrac{89}{252}\right) \chi _A^2
+\left(\tfrac{10}{9}\eta^2 -\tfrac{1817}{504}\eta +\tfrac{89}{252}\right) \chi _S^2 \Big] \nonumber\\
&\quad
+ v_\Omega^7 \Big[
\delta  \left(\tfrac{97865}{63504}\eta^2 +\tfrac{50140}{3969}\eta +\tfrac{18733}{15876}\right) \chi _A
+\left(\tfrac{50803}{63504}\eta^3 -\tfrac{245717}{63504}\eta^2 +\tfrac{74749 }{5292}\eta +\tfrac{18733}{15876}\right) \chi _S
{+ \delta \chi _A^3 \left(\tfrac{1}{3}-\tfrac{4}{3}\eta\right)+\delta  (2 \eta +1) \chi _A \chi _S^2} \nonumber\\
&\qquad
{+ \left(-4 \eta ^2-3 \eta +1\right) \chi _A^2 \chi _S+\left(\eta +\tfrac{1}{3}\right) \chi _S^3}\Big] 
+ v_\Omega^8 \left[\tfrac{9202}{2205}\text{eulerlog}(2,v_\Omega)-\tfrac{387216563023}{160190110080}\right]\nonumber \\
&\quad 
+ v_\Omega^{10} \left[\tfrac{439877}{55566}\text {eulerlog}(2,v_\Omega)-\tfrac{16094530514677}{533967033600}\right], \\
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho22 = 1 + 
    vomega**2 * (sp.Rational(55, 84) * eta - sp.Rational(43, 42)) + 
    vomega**3 * ((sp.Rational(2, 3) * eta - sp.Rational(2, 3)) * chi_S - sp.Rational(2, 3) * delta * chi_A) +
    vomega**4 * (sp.Rational(19583, 42336) * eta**2 - sp.Rational(33025, 21168) * eta - sp.Rational(20555, 10584) +
                (sp.Rational(1, 2) - 2 * eta) * chi_A**2 + delta * chi_A * chi_S + sp.Rational(1, 2) * chi_S**2) + 
    vomega**5 * (delta * (sp.Rational(-19, 42) * eta - sp.Rational(34, 21)) * chi_A +
                (sp.Rational(209, 126) * eta**2 + sp.Rational(49, 18) * eta - sp.Rational(34, 21)) * chi_S) + 
    vomega**6 * (sp.Rational(10620745, 39118464) * eta**3 - sp.Rational(6292061, 3259872) * eta**2 +
                sp.Rational(41, 192) * sp.pi**2 * eta - sp.Rational(48993925, 9779616) * eta -
                sp.Rational(428, 105) * eulerlog2 + sp.Rational(1556919113, 122245200) +
                delta * (sp.Rational(89, 126) - sp.Rational(781, 252) * eta) * chi_A * chi_S +
                (sp.Rational(-27, 14) * eta**2 - sp.Rational(457, 504) * eta + sp.Rational(89, 252)) * chi_A**2 +
                (sp.Rational(10, 9) * eta**2 - sp.Rational(1817, 504) * eta + sp.Rational(89, 252)) * chi_S**2) + 
    vomega**7 * (delta * (sp.Rational(97865, 63504) * eta**2 + sp.Rational(50140, 3969) * eta + sp.Rational(18733, 15876)) * chi_A +
                (sp.Rational(50803, 63504) * eta**3 - sp.Rational(245717, 63504) * eta**2 +
                sp.Rational(74749, 5292) * eta + sp.Rational(18733, 15876)) * chi_S +
                delta * chi_A**3 * (sp.Rational(1, 3) - sp.Rational(4, 3) * eta) +
                delta * (2 * eta + 1) * chi_A * chi_S**2 +
                (sp.Rational(-4, 1) * eta**2 - sp.Rational(3, 1) * eta + sp.Rational(1, 1)) * chi_A**2 * chi_S +
                (eta + sp.Rational(1, 3)) * chi_S**3) + 
    vomega**8 * (sp.Rational(9202, 2205) * eulerlog2 - sp.Rational(387216563023, 160190110080)) + 
    vomega**10 * (sp.Rational(439877, 55566) * eulerlog2 - sp.Rational(16094530514677, 533967033600))

The (2,1) mode is given in Equation B2a:

\begin{align*}
\rho_{21}^\text{NS} &=1
+v_\Omega^2 \left(\tfrac{23}{84}\eta -\tfrac{59}{56}\right) 
+v_\Omega^4 \left(\tfrac{617}{4704}\eta^2 -\tfrac{10993}{14112}\eta -\tfrac{47009}{56448}\right)  +v_\Omega^6 \left[\tfrac{7613184941}{2607897600}-\tfrac{107}{105} \text { eulerlog }\left(1, v_\Omega\right)\right]  \nonumber \\
&\quad+ v_\Omega^8\left[-\tfrac{1168617463883}{911303737344}+\tfrac{6313}{5880}\text { eulerlog }\left(1, v_\Omega\right)\right]
+v_\Omega^{10}\left[-\tfrac{63735873771463}{16569158860800} + \tfrac{5029963}{5927040} \text{ eulerlog }\left(1, v_\Omega\right)\right] ,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho21NS = 1 + 
    vomega**2 * (sp.Rational(23, 84) * eta - sp.Rational(59, 56)) + 
    vomega**4 * (sp.Rational(617, 4704) * eta**2 - sp.Rational(10993, 14112) * eta - sp.Rational(47009, 56448)) + 
    vomega**6 * (sp.Rational(7613184941, 2607897600) - sp.Rational(107, 105) * eulerlog1) + 
    vomega**8 * (-sp.Rational(1168617463883, 911303737344) + sp.Rational(6313, 5880) * eulerlog1) + 
    vomega**10 * (-sp.Rational(63735873771463, 16569158860800) + sp.Rational(5029963, 5927040) * eulerlog1)

The (3,3) mode is given in Equation B3a:

\begin{align*}
\rho_{33}^\text{NS}&= 1+v_\Omega^2 \left(\tfrac{2}{3}\eta-\tfrac{7}{6}\right)  + v_\Omega^4\left(-\tfrac{6719}{3960}-\tfrac{1861}{990}\eta+\tfrac{149 }{330}\eta v_\Omega^2\right) \nonumber \\
&\quad
+v_\Omega^6 \left[\tfrac{3203101567}{227026800}+\left(-\tfrac{129509}{25740}+\tfrac{41 \pi^2}{192}\right) \eta-\tfrac{274621}{154440} \eta^2+\tfrac{12011 }{46332}\eta^3-\tfrac{26}{7} \text{eulerlog}\left(3, v_\Omega\right)\right]  \nonumber \\
&\quad
+v_\Omega^8\left[-\tfrac{57566572157}{8562153600}+\tfrac{13}{3} \text { eulerlog }\left(3, v_\Omega\right)\right] 
+v_\Omega^{10}\left[-\tfrac{903823148417327}{30566888352000}+\tfrac{87347}{13860}\text{eulerlog}\left(3, v_\Omega\right)\right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho33NS = 1 + 
    vomega**2 * (sp.Rational(2, 3) * eta - sp.Rational(7, 6)) + 
    vomega**4 * (-sp.Rational(6719, 3960) - sp.Rational(1861, 990) * eta + sp.Rational(149, 330) * eta * vomega**2) + 
    vomega**6 * (sp.Rational(3203101567, 227026800) + (-sp.Rational(129509, 25740) + sp.Rational(41, 192) * sp.pi**2) * eta -
                sp.Rational(274621, 154440) * eta**2 + sp.Rational(12011, 46332) * eta**3 - sp.Rational(26, 7) * eulerlog3) + 
    vomega**8 * (-sp.Rational(57566572157, 8562153600) + sp.Rational(13, 3) * eulerlog3) + 
    vomega**10 * (-sp.Rational(903823148417327, 30566888352000) + sp.Rational(87347, 13860) * eulerlog3)

The (3,2) mode is given in Equation B6a:

\begin{align*}
\rho_{32} &= 1 + v_\Omega \frac{4 \eta \chi _S}{3 (1-3 \eta )}
+ v_\Omega^2 \left[
\frac{-\tfrac{32}{27}\eta ^2+\tfrac{223}{54}\eta-\tfrac{164}{135}}{1-3 \eta }
{-\frac{16 \eta ^2 \chi _S^2}{9 (1-3 \eta )^2}}
\right] \nonumber\\
&\quad
+ v_\Omega^3 \left[{\left(\tfrac{13 }{9}\eta +\tfrac{2}{9}\right) \frac{\delta \chi _A}{1-3 \eta }
+\left(\tfrac{607}{81}\eta ^3 +\tfrac{503}{81}\eta ^2 -\tfrac{1478}{405}\eta +\tfrac{2}{9}\right)  \frac{\chi _S}{(1-3 \eta )^2} } 
{+\frac{320 \eta ^3 \chi _S^3}{81 (1-3 \eta )^3}}\right]\nonumber\\
&\quad
+ v_\Omega^4 \Bigg[
\frac{\tfrac{77141}{40095} \eta ^4 -\tfrac{508474 }{40095}\eta ^3 -\tfrac{945121 }{320760}\eta ^2 +\tfrac{1610009 \eta }{320760}-\tfrac{180566}{200475}}{(1-3 \eta )^2}
+ {\left(4 \eta ^2-3 \eta +\tfrac{1}{3}\right) \frac{\chi _A^2}{1-3 \eta }
+ \left(-\tfrac{50}{27}\eta ^2-\tfrac{88}{27}\eta+\tfrac{2}{3}\right) \frac{\delta \chi _A \chi _S}{(1-3 \eta )^2} } \nonumber\\
&\qquad
{
+ \left(-\tfrac{2452}{243} \eta ^4 -\tfrac{1997}{243} \eta ^3 +\tfrac{1435}{243}\eta ^2 -\tfrac{43}{27}\eta +\tfrac{1}{3}\right)  \frac{\chi _S^2}{(1-3 \eta )^3}}
\Bigg] \nonumber\\
&\quad
+ {v_\Omega^5 \Bigg[
\left(-\tfrac{1184225 }{96228}\eta ^5 -\tfrac{40204523}{962280} \eta ^4 +\tfrac{101706029 }{962280}\eta ^3 -\tfrac{14103833 }{192456}\eta ^2 +\tfrac{20471053}{962280}\eta -\tfrac{2788}{1215}\right)  \frac{\chi _S}{(1-3 \eta )^3}
+ \left(\tfrac{608}{81} \eta ^3+\tfrac{736}{81} \eta ^2 -\tfrac{16  }{9}\eta\right)  \frac{\delta \chi _A \chi _S^2}{(1-3 \eta )^3}} \nonumber\\
&\qquad
{+\left(\tfrac{889673}{106920}\eta ^3-\tfrac{75737}{5346} \eta ^2+\tfrac{376177 }{35640}\eta -\tfrac{2788}{1215}\right)  \frac{\delta  \chi _A}{(1-3 \eta )^2}
+\left(\tfrac{96176 }{2187}\eta ^5 +\tfrac{43528}{2187} \eta ^4-\tfrac{40232 }{2187}\eta ^3 +\tfrac{376 }{81}\eta ^2 -\tfrac{8 \eta }{9}\right)  \frac{\chi _S^3}{(1-3 \eta )^4}} \nonumber\\
&\qquad
{+\left(-\tfrac{32 }{3}\eta ^3+8 \eta ^2-\tfrac{8}{9}\eta\right)  \frac{\chi _A^2 \chi _S}{(1-3 \eta )^2} \Bigg]}
+v_\Omega^6\left[\tfrac{5849948554}{940355325}-\tfrac{104 \text { eulerlog}(2,v_\Omega)}{63}\right] 
+v_\Omega^8 \left[\tfrac{17056 \text { eulerlog}(2,v_\Omega)}{8505}-\tfrac{10607269449358}{3072140846775}\right] \nonumber \\
&\qquad
+ {v_\Omega^{10}\left[- \tfrac{1312549797426453052}{176264081083715625} +\tfrac{18778864 \text { eulerlog}(2,v_{\Omega} )}{12629925}   \right]},
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho32 = 1 + 
    vomega * (4 * eta * chi_S) / (3 * (1 - 3 * eta)) + 
    vomega**2 * ((sp.Rational(-32, 27) * eta**2 + sp.Rational(223, 54) * eta - sp.Rational(164, 135)) / (1 - 3 * eta) -
                (16 * eta**2 * chi_S**2) / (9 * (1 - 3 * eta)**2)) + 
    vomega**3 * ((sp.Rational(13, 9) * eta + sp.Rational(2, 9)) * (delta * chi_A) / (1 - 3 * eta) +
                (sp.Rational(607, 81) * eta**3 + sp.Rational(503, 81) * eta**2 - sp.Rational(1478, 405) * eta + sp.Rational(2, 9)) * chi_S / (1 - 3 * eta)**2 +
                (320 * eta**3 * chi_S**3) / (81 * (1 - 3 * eta)**3)) + 
    vomega**4 * ((sp.Rational(77141, 40095) * eta**4 - sp.Rational(508474, 40095) * eta**3 - sp.Rational(945121, 320760) * eta**2 +
                sp.Rational(1610009, 320760) * eta - sp.Rational(180566, 200475)) / (1 - 3 * eta)**2 +
                (4 * eta**2 - 3 * eta + sp.Rational(1, 3)) * (chi_A**2) / (1 - 3 * eta) +
                (sp.Rational(-50, 27) * eta**2 - sp.Rational(88, 27) * eta + sp.Rational(2, 3)) * (delta * chi_A * chi_S) / (1 - 3 * eta)**2 +
                (sp.Rational(-2452, 243) * eta**4 - sp.Rational(1997, 243) * eta**3 + sp.Rational(1435, 243) * eta**2 -
                sp.Rational(43, 27) * eta + sp.Rational(1, 3)) * (chi_S**2) / (1 - 3 * eta)**3)) + 
    vomega**5 * ((sp.Rational(-1184225, 96228) * eta**5 - sp.Rational(40204523, 962280) * eta**4 + sp.Rational(101706029, 962280) * eta**3 -
                sp.Rational(14103833, 192456) * eta**2 + sp.Rational(20471053, 962280) * eta - sp.Rational(2788, 1215)) * chi_S / (1 - 3 * eta)**3 +
                (sp.Rational(608, 81) * eta**3 + sp.Rational(736, 81) * eta**2 - sp.Rational(16, 9) * eta) * (delta * chi_A * chi_S**2) / (1 - 3 * eta)**3 +
                (sp.Rational(889673, 106920) * eta**3 - sp.Rational(75737, 5346) * eta**2 + sp.Rational(376177, 35640) * eta -
                sp.Rational(2788, 1215)) * (delta * chi_A) / (1 - 3 * eta)**2 +
                (sp.Rational(96176, 2187) * eta**5 + sp.Rational(43528, 2187) * eta**4 - sp.Rational(40232, 2187) * eta**3 +
                sp.Rational(376, 81) * eta**2 - sp.Rational(8, 9) * eta) * (chi_S**3) / (1 - 3 * eta)**4 +
                (sp.Rational(-32, 3) * eta**3 + 8 * eta**2 - sp.Rational(8, 9) * eta) * (chi_A**2 * chi_S) / (1 - 3 * eta)**2)) + 
    vomega**6 * (sp.Rational(5849948554, 940355325) - sp.Rational(104, 63) * eulerlog2) + 
    vomega**8 * (sp.Rational(17056, 8505) * eulerlog2 - sp.Rational(10607269449358, 3072140846775)) + 
    vomega**10 * (-sp.Rational(1312549797426453052, 176264081083715625) + sp.Rational(18778864, 12629925) * eulerlog2)

The (3,1) mode is given in Equation B8:

\begin{align*}
\rho_{31}^\text{NS} &= 1-v_\Omega^2\left[\frac{2}{9}\eta+\frac{13}{18}\right] 
+ v_\Omega^4  \left[-\frac{829}{1782}\eta^2-\frac{1685}{1782}\eta+\frac{101}{7128}\right] 
+ v_\Omega^6 \left[\frac{11706720301}{6129723600}-\frac{26}{63}\text{eulerlog}(1,v_\Omega^2)\right] \nonumber\\
&\quad
+ v_\Omega^8 \left[\frac{169}{567}\text{eulerlog}(1,v_\Omega^2) +\frac{2606097992581}{4854741091200}\right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho31NS = 1 - 
    vomega**2 * (sp.Rational(2, 9) * eta + sp.Rational(13, 18)) + 
    vomega**4 * (-sp.Rational(829, 1782) * eta**2 - sp.Rational(1685, 1782) * eta + sp.Rational(101, 7128)) + 
    vomega**6 * (sp.Rational(11706720301, 6129723600) - sp.Rational(26, 63) * eulerlog1v2) + 
    vomega**8 * (sp.Rational(169, 567) * eulerlog1v2 + sp.Rational(2606097992581, 4854741091200))

The (4,4) mode is given in Equation B:

\begin{align*}
\rho_{44} &= 1+v_\Omega^2\left[\tfrac{1614-5870 \eta +2625 \eta ^2}{1320 (-1+3 \eta )}\right] + v_\Omega^3 \left[\left(\tfrac{2}{3}-\tfrac{41 \eta }{15}+\tfrac{14 \eta ^2}{5}\right) \tfrac{1}{(-1+3 \eta )}\chi_S 
+\delta \left(\tfrac{2}{3}-\tfrac{13 \eta }{5}\right) \tfrac{1}{(-1+3 \eta )}\chi _A\right]  \nonumber \\
&\quad
+v_\Omega^4\left[-\tfrac{14210377}{8808800 (1-3 \eta )^2}+\tfrac{32485357 \eta
}{4404400 (1-3 \eta )^2}-\tfrac{1401149 \eta ^2}{1415700 (1-3 \eta )^2} 
-\tfrac{801565 \eta ^3}{37752 (1-3 \eta )^2}+\tfrac{3976393 \eta ^4}{1006720 (1-3 \eta )^2}+\tfrac{1}{2}\chi _A^2-2 \eta  \chi _A^2+\delta \chi _A \chi _S+\tfrac{1}{2}\chi _S^2\right]  \nonumber \\
&\quad+ v_\Omega^5\Bigg[\left(-\tfrac{69}{55}+\tfrac{16571 \eta }{1650}-\tfrac{2673 \eta ^2}{100}+\tfrac{8539 \eta ^3}{440}+\tfrac{591 \eta ^4}{44}\right) \tfrac{1}{(1-3 \eta)^2}\chi _S
+\delta
\left(-\tfrac{69}{55}+\tfrac{10679 \eta }{1650}-\tfrac{1933 \eta ^2}{220}+\tfrac{597 \eta ^3}{440}\right) \tfrac{1}{ (1-3 \eta )^2}\chi_A\Bigg]  \nonumber \\
&\quad+v_\Omega^6 \left[\tfrac{16600939332793}{1098809712000}-\tfrac{12568 }{3465}\text{eulerlog}\left(4,v_\Omega\right)\right]  +v_\Omega^8\left[-\tfrac{172066910136202271}{19426955708160000}+\tfrac{845198 }{190575}\text{eulerlog}\left(4,v_\Omega\right)\right] \nonumber \\
&\quad+ v_\Omega^{10}\left[-\tfrac{17154485653213713419357}{568432724020761600000}+\tfrac{22324502267}{3815311500} \text{eulerlog}\left(4,v_\Omega\right)\right]\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho44 = 1 + 
    vomega**2 * ((1614 - 5870 * eta + 2625 * eta**2) / (1320 * (-1 + 3 * eta))) + 
    vomega**3 * ((sp.Rational(2, 3) - sp.Rational(41, 15) * eta + sp.Rational(14, 5) * eta**2) / (-1 + 3 * eta) * chi_S +
                delta * (sp.Rational(2, 3) - sp.Rational(13, 5) * eta) / (-1 + 3 * eta) * chi_A) + 
    vomega**4 * (-sp.Rational(14210377, 8808800) / (1 - 3 * eta)**2 + sp.Rational(32485357, 4404400) * eta / (1 - 3 * eta)**2 -
                sp.Rational(1401149, 1415700) * eta**2 / (1 - 3 * eta)**2 -
                sp.Rational(801565, 37752) * eta**3 / (1 - 3 * eta)**2 +
                sp.Rational(3976393, 1006720) * eta**4 / (1 - 3 * eta)**2 +
                (sp.Rational(1, 2) * chi_A**2 - 2 * eta * chi_A**2 + delta * chi_A * chi_S + sp.Rational(1, 2) * chi_S**2)) + 
    vomega**5 * ((-sp.Rational(69, 55) + sp.Rational(16571, 1650) * eta - sp.Rational(2673, 100) * eta**2 +
                sp.Rational(8539, 440) * eta**3 + sp.Rational(591, 44) * eta**4) / (1 - 3 * eta)**2 * chi_S +
                delta * (-sp.Rational(69, 55) + sp.Rational(10679, 1650) * eta - sp.Rational(1933, 220) * eta**2 +
                sp.Rational(597, 440) * eta**3) / (1 - 3 * eta)**2 * chi_A) + 
    vomega**6 * (sp.Rational(16600939332793, 1098809712000) - sp.Rational(12568, 3465) * eulerlog4) + 
    vomega**8 * (-sp.Rational(172066910136202271, 19426955708160000) + sp.Rational(845198, 190575) * eulerlog4) + 
    vomega**10 * (-sp.Rational(17154485653213713419357, 568432724020761600000) + sp.Rational(22324502267, 3815311500) * eulerlog4)

The (4,3) mode is given in Equation B:

\begin{align*}
\rho_{43}^\text{NS} &= 1 + \frac{v_\Omega^2}{1-2 \eta } \left(-\tfrac{10}{11} \eta ^2+\tfrac{547}{176}\eta-\tfrac{111}{88}\right) -\tfrac{6894273}{7047040} v_\Omega^4
+v_\Omega^6\left[\tfrac{1664224207351}{195343948800}-\tfrac{1571}{770} \text { eulerlog}(3,v_\Omega)\right] \nonumber \\
&\quad
+  v_\Omega^8\left[-\tfrac{2465107182496333}{460490801971200}  + \tfrac{174381}{67760} \text { eulerlog}(3,v_{\Omega} )   \right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS43 = 1 + 
    vomega**2 / (1 - 2 * eta) * (-sp.Rational(10, 11) * eta**2 + sp.Rational(547, 176) * eta - sp.Rational(111, 88)) - 
    sp.Rational(6894273, 7047040) * vomega**4 + 
    vomega**6 * (sp.Rational(1664224207351, 195343948800) - sp.Rational(1571, 770) * eulerlog3) + 
    vomega**8 * (-sp.Rational(2465107182496333, 460490801971200) + sp.Rational(174381, 67760) * eulerlog3)

The (4,2) mode is given in Equation B:

\begin{align*}
\rho_{42}&=1+\frac{285\,\eta^2-3530\,\eta+1146}{1320\,(3\,\eta-1)} v_\Omega^2 - \frac{v_\Omega^3}{15(1-3\eta)} \left[(78\eta^2 - 59\eta + 10)\chi_S +(10-21\eta)\delta\,\chi_A \right] \nonumber \\
&\quad+\frac{-379526805\,\eta^4-3047981160\,\eta^3+1204388696\,\eta^2+295834536\,\eta-114859044}{317116800\,(1-3\,\eta)^2} v_\Omega^4 \nonumber\\
&\quad+ \left[\frac{848238724511}{219761942400}-\frac{3142}{3465}\text{eulerlog}(2,v_\Omega^2)\right] v_\Omega^6\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho42 = 1 + 
    ((285 * eta**2 - 3530 * eta + 1146) / (1320 * (3 * eta - 1))) * vomega**2 - 
    (vomega**3 / (15 * (1 - 3 * eta))) * ((78 * eta**2 - 59 * eta + 10) * chi_S + (10 - 21 * eta) * delta * chi_A) - 
    ((-379526805 * eta**4 - 3047981160 * eta**3 + 1204388696 * eta**2 + 295834536 * eta - 114859044) /
    (317116800 * (1 - 3 * eta)**2)) * vomega**4 + 
    ((848238724511 / 219761942400) - (3142 / 3465) * eulerlog2v2) * vomega**6

The (4,1) mode is given in Equation B:

\begin{align*}
\rho_{41}^\text{NS} &= 1 +\frac{288\,\eta^2-1385\,\eta+602}{528\,(2\,\eta-1)}v_\Omega^2
-\frac{7775491}{21141120} v_\Omega^4 + \left[\frac{1227423222031}{1758095539200}-\frac{1571}{6930}\text{eulerlog}(1,v_\Omega^2)\right]\, v_\Omega^6 \,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS41 = 1 + 
    ((288 * eta**2 - 1385 * eta + 602) / (528 * (2 * eta - 1))) * vomega**2 - 
    (sp.Rational(7775491, 21141120)) * vomega**4 + 
    ((sp.Rational(1227423222031, 1758095539200) - sp.Rational(1571, 6930) * eulerlog1v2) * vomega**6)

The (5,5) mode is given in Equation B:

\begin{align*}
\rho_{55}^{\mathrm{NS}} =& 1+v_\Omega^2\left[\tfrac{487}{390 (-1+2 \eta )}-\tfrac{649 \eta }{195 (-1+2 \eta )}+\tfrac{256 \eta ^2}{195 (-1+2 \eta )}\right] -\tfrac{3353747}{2129400} v_\Omega^4 \nonumber \\
   &+v_\Omega^6\left[\tfrac{190606537999247}{11957879934000}-\tfrac{1546}{429}\text{eulerlog}\left(5,v_\Omega\right) \right] +v_\Omega^8 \left[-\tfrac{1213641959949291437}{118143853747920000}+\tfrac{376451 }{83655}\text{eulerlog}\left(5,v_\Omega\right)\right] \nonumber \\
   &+v_\Omega^{10}\left[-\tfrac{150082616449726042201261}{4837990810977324000000}+\tfrac{2592446431}{456756300}\text{eulerlog}\left(5,v_\Omega\right)\right]\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS55 = 1 + \
    vomega**2 * (sp.Rational(487, 390) / (-1 + 2 * eta) - sp.Rational(649, 195) * eta / (-1 + 2 * eta) +
                 sp.Rational(256, 195) * eta**2 / (-1 + 2 * eta)) - \
    sp.Rational(3353747, 2129400) * vomega**4 + \
    (sp.Rational(190606537999247, 11957879934000) - sp.Rational(1546, 429) * eulerlog5) * vomega**6 + \
    (-sp.Rational(1213641959949291437, 118143853747920000) + sp.Rational(376451, 83655) * eulerlog5) * vomega**8 + \
    (-sp.Rational(150082616449726042201261, 4837990810977324000000) + sp.Rational(2592446431, 456756300) * eulerlog5) * vomega**10


The remaining $l = 5$ modes are given in Equation B:

\begin{align*}
\rho_{54}&=1+\frac{33320\,\eta^3-127610\,\eta^2+96019\,\eta-17448}{13650\,(5\,\eta^2-5\,\eta+1)} v_\Omega^2 -\frac{16213384}{15526875} v_\Omega^4 \,, \\
%%(5,3)
\rho^{\rm NS}_{53}&=1+\frac{176\,\eta^2-850\,\eta+375}{390\,(2\,\eta-1)} v_\Omega^2 -\frac{410833}{709800} v_\Omega^4\,, \\
%%(5,2)
\rho_{52}&=1+\frac{21980\,\eta^3-104930\,\eta^2+84679\,\eta-15828}{13650\,(5\,\eta^2-5\,\eta+1)} v_\Omega^2 -\frac{7187914}{15526875} v_\Omega^4 \,, \\
%%(5,1)
\rho^{\rm NS}_{51}&=1+\frac{8\,\eta^2-626\,\eta+319}{390\,(2\,\eta-1)}v_\Omega^2 -\frac{31877}{304200} v_\Omega^4\,, \\
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho54 = 1 + 
    ((33320 * eta**3 - 127610 * eta**2 + 96019 * eta - 17448) /
    (13650 * (5 * eta**2 - 5 * eta + 1))) * vomega**2 - 
    sp.Rational(16213384, 15526875) * vomega**4

rhoNS53 = 1 + 
    ((176 * eta**2 - 850 * eta + 375) / (390 * (2 * eta - 1))) * vomega**2 - 
    sp.Rational(410833, 709800) * vomega**4

rho52 = 1 + 
    ((21980 * eta**3 - 104930 * eta**2 + 84679 * eta - 15828) /
    (13650 * (5 * eta**2 - 5 * eta + 1))) * vomega**2 - 
    sp.Rational(7187914, 15526875) * vomega**4

rhoNS51 = 1 + 
    ((8 * eta**2 - 626 * eta + 319) / (390 * (2 * eta - 1))) * vomega**2 - 
    sp.Rational(31877, 304200) * vomega**4

The $l = 6$ modes are given in Equation B:

\begin{align*}
\rho_{66} &= 1+\frac{273\,\eta^3-861\,\eta^2+602\,\eta-106}{84\,(5\,\eta^2-5\,\eta+1)} v_\Omega^2 - \frac{1025435}{659736} v_\Omega^4 \,, \\
%%(6,5)
\rho_{65}& = 1 + \frac{220\,\eta^3-910\,\eta^2+838\,\eta-185}{144\,(3\,\eta^2-4\,\eta+1)} v_\Omega^2\,, \\
%%(6,4)
\rho_{64} &= 1 + \frac{133\,\eta^3-581\,\eta^2+462\,\eta-86}{84\,(5\,\eta^2-5\,\eta+1)} v_\Omega^2 - \frac{476887}{659736} v_\Omega^4 \,, \\
%%(6,3)
\rho_{63} &= 1 + \frac{156\,\eta^3-750\,\eta^2+742\,\eta-169}{144\,(3\,\eta^2-4\,\eta+1)} v_\Omega^2\,, \\
%%(6,2)
\rho_{62} &= 1 + \frac{49\,\eta^3-413\,\eta^2+378\,\eta-74}{84\,(5\,\eta^2-5\,\eta+1)} v_\Omega^2 - \frac{817991}{3298680} v_\Omega^4 \,, \\
%%(6,1)
\rho_{61} &= 1 + \frac{124\,\eta^3-670\,\eta^2+694\,\eta-161}{144\,(3\,\eta^2-4\,\eta+1)} v_\Omega^2\,, \\
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho66 = 1 + 
    ((273 * eta**3 - 861 * eta**2 + 602 * eta - 106) /
    (84 * (5 * eta**2 - 5 * eta + 1))) * vomega**2 - 
    sp.Rational(1025435, 659736) * vomega**4

rhoNS65 = 1 + 
    ((220 * eta**3 - 910 * eta**2 + 838 * eta - 185) /
    (144 * (3 * eta**2 - 4 * eta + 1))) * vomega**2

rho64 = 1 + 
    ((133 * eta**3 - 581 * eta**2 + 462 * eta - 86) /
    (84 * (5 * eta**2 - 5 * eta + 1))) * vomega**2 - 
    sp.Rational(476887, 659736) * vomega**4

rhoNS63 = 1 + 
    ((156 * eta**3 - 750 * eta**2 + 742 * eta - 169) /
    (144 * (3 * eta**2 - 4 * eta + 1))) * vomega**2

rho62 = 1 + 
    ((49 * eta**3 - 413 * eta**2 + 378 * eta - 74) /
    (84 * (5 * eta**2 - 5 * eta + 1))) * vomega**2 - 
    sp.Rational(817991, 3298680) * vomega**4

rhoNS61 = 1 + 
    ((124 * eta**3 - 670 * eta**2 + 694 * eta - 161) /
    (144 * (3 * eta**2 - 4 * eta + 1))) * vomega**2

The $l = 7$ modes are given in Equation B:

\begin{align*}
%%(7,7)
\rho_{77} &= 1 + \frac{1380 \eta ^3-4963 \eta ^2+4246 \eta -906}{714 \left(3 \eta ^2-4 \eta +1\right)} v_{\Omega }^2 \,, \\
%%(7,6)
\rho_{76} &= 1 + \frac{6104 \eta ^4-29351 \eta ^3+37828 \eta^2-16185 \eta +2144} {1666 \left(7 \eta ^3-14 \eta ^2+7 \eta-1\right)} v_{\Omega }^2 \,, \\
%%(7,5)
\rho_{75} &= 1 + \frac{804 \eta ^3-3523 \eta ^2+3382 \eta -762}{714 \left(3 \eta ^2-4 \eta +1\right)} v_{\Omega }^2 \,, \\
%%(7,4)
\rho_{74} &= 1 + \frac{41076 \eta ^4-217959 \eta ^3+298872 \eta^2-131805 \eta +17756} {14994 \left(7 \eta ^3-14 \eta ^2+7 \eta-1\right)} v_{\Omega }^2 \,, \\
%%(7,3)
\rho_{73} &= 1 + \frac{420 \eta ^3-2563 \eta ^2+2806 \eta -666}{714 \left(3 \eta ^2-4 \eta +1\right)} v_{\Omega }^2 \,, \\
%%(7,2)
\rho_{72} &= 1 + \frac{32760 \eta ^4-190239 \eta ^3+273924 \eta^2-123489 \eta +16832} {14994 \left(7 \eta ^3-14 \eta ^2+7 \eta-1\right)} v_{\Omega }^2 \,, \\
%%(7,1)
\rho_{71} &= 1 + \frac{228 \eta ^3-2083 \eta ^2+2518 \eta -618}{714 \left(3 \eta ^2-4 \eta +1\right)} v_{\Omega }^2 \,, \\
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS77 = 1 + 
    ((1380 * eta**3 - 4963 * eta**2 + 4246 * eta - 906) /
    (714 * (3 * eta**2 - 4 * eta + 1))) * vomega**2

rho76 = 1 + 
    ((6104 * eta**4 - 29351 * eta**3 + 37828 * eta**2 - 16185 * eta + 2144) /
    (1666 * (7 * eta**3 - 14 * eta**2 + 7 * eta - 1))) * vomega**2

rhoNS75 = 1 + 
    ((804 * eta**3 - 3523 * eta**2 + 3382 * eta - 762) /
    (714 * (3 * eta**2 - 4 * eta + 1))) * vomega**2

rhoNS74 = 1 + 
    ((41076 * eta**4 - 217959 * eta**3 + 298872 * eta**2 - 131805 * eta + 17756) /
    (14994 * (7 * eta**3 - 14 * eta**2 + 7 * eta - 1))) * vomega**2

rhoNS73 = 1 + 
    ((420 * eta**3 - 2563 * eta**2 + 2806 * eta - 666) /
    (714 * (3 * eta**2 - 4 * eta + 1))) * vomega**2

rhoNS72 = 1 + 
    ((32760 * eta**4 - 190239 * eta**3 + 273924 * eta**2 - 123489 * eta + 16832) /
    (14994 * (7 * eta**3 - 14 * eta**2 + 7 * eta - 1))) * vomega**2

rhoNS71 = 1 + 
    ((228 * eta**3 - 2083 * eta**2 + 2518 * eta - 618) /
    (714 * (3 * eta**2 - 4 * eta + 1))) * vomega**2

The $l = 8$ modes are given in Equation B:

\begin{align*}
%%(8,8)
\rho_{88} &= 1+ \frac{3482 - 26778\eta + 64659\eta^2 - 53445\eta^3 + 12243\eta^4}{2736 (-1 + 7 \eta- 14 \eta^2 + 7 \eta^3)} v_\Omega^2\,, \\
%%(8,7)
\rho_{87} &= 1+ \frac{23478 - 154099\eta + 309498\eta^2 - 207550\eta^3 + 38920\eta^4}{18240 (-1 + 6\eta - 10\eta^2 + 4\eta^3)} v_\Omega^2\,, \\
%%(8,6)
\rho_{86} &= 1+ \frac{1002 - 7498\eta + 17269\eta^2 - 13055\eta^3 + 2653\eta^4}{912(-1 + 7\eta - 14\eta^2 + 7\eta^3)} v_\Omega^2\,, \\
%%(8,5)
\rho_{85} &= 1+ \frac{4350 - 28055\eta + 54642\eta^2 - 34598\eta^3 + 6056\eta^4}{3648 (-1 + 6\eta - 10\eta^2 + 4\eta^3)} v_\Omega^2\,, \\
%%(8,4)
\rho_{84} &= 1+ \frac{2666 - 19434\eta + 42627\eta^2 - 28965\eta^3 + 4899\eta^4}{2736(-1 + 7\eta - 14\eta^2 + 7\eta^3)} v_\Omega^2\,, \\
%%(8,3)
\rho_{83} &= 1+ \frac{20598 - 131059\eta + 249018\eta^2 - 149950\eta^3 + 24520\eta^4}{18240(-1 + 6\eta - 10\eta^2 + 4\eta^3)} v_\Omega^2\,, \\
%%(8,2)
\rho_{82} &= 1+ \frac{2462 - 17598\eta + 37119\eta^2 - 22845\eta^3 + 3063\eta^4}{2736(-1 + 7\eta - 14\eta^2 + 7\eta^3)} v_\Omega^2\,, \\
%%(8,1)
\rho_{81} &= 1+ \frac{20022 - 126451\eta + 236922\eta^2 - 138430\eta^3 + 21640\eta^4}{18240(-1 + 6\eta - 10\eta^2 + 4\eta^3)} v_\Omega^2\,.
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho88 = 1 + ((3482 - 26778*eta + 64659*eta**2 - 53445*eta**3 + 12243*eta**4) /
              (2736 * (-1 + 7*eta - 14*eta**2 + 7*eta**3))) * vomega**2

rhoNS87 = 1 + ((23478 - 154099*eta + 309498*eta**2 - 207550*eta**3 + 38920*eta**4) /
              (18240 * (-1 + 6*eta - 10*eta**2 + 4*eta**3))) * vomega**2

rho86 = 1 + ((1002 - 7498*eta + 17269*eta**2 - 13055*eta**3 + 2653*eta**4) /
              (912 * (-1 + 7*eta - 14*eta**2 + 7*eta**3))) * vomega**2

rhoNS85 = 1 + ((4350 - 28055*eta + 54642*eta**2 - 34598*eta**3 + 6056*eta**4) /
              (3648 * (-1 + 6*eta - 10*eta**2 + 4*eta**3))) * vomega**2

rho84 = 1 + ((2666 - 19434*eta + 42627*eta**2 - 28965*eta**3 + 4899*eta**4) /
              (2736 * (-1 + 7*eta - 14*eta**2 + 7*eta**3))) * vomega**2

rhoNS83 = 1 + ((20598 - 131059*eta + 249018*eta**2 - 149950*eta**3 + 24520*eta**4) /
              (18240 * (-1 + 6*eta - 10*eta**2 + 4*eta**3))) * vomega**2

rho82 = 1 + ((2462 - 17598*eta + 37119*eta**2 - 22845*eta**3 + 3063*eta**4) /
              (2736 * (-1 + 7*eta - 14*eta**2 + 7*eta**3))) * vomega**2

rhoNS81 = 1 + ((20022 - 126451*eta + 236922*eta**2 - 138430*eta**3 + 21640*eta**4) /
              (18240 * (-1 + 6*eta - 10*eta**2 + 4*eta**3))) * vomega**2

<a id='fspin'></a>

# Step : $f^{\rm S}_{lm}$ \[Back to [top](#toc)\]
$$\label{fspin}$$

$f^{\rm S}_{lm}$ is defined in Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf).

We begin with the (2,1) mode given in Equation B2b:

\begin{align*}
f^{\rm S}_{21} &= -\tfrac{3}{2} v_\Omega \left(\frac{\chi _A}{\delta} +\chi _S\right)
+ v_\Omega^3 \Big[\left(\tfrac{131}{84}\eta +\tfrac{61}{12}\right) \frac{\chi _A}{\delta} +\left(\tfrac{79}{84}\eta +\tfrac{61}{12}\right) \chi _S\Big]
+ v_\Omega^4 \left[(-2 \eta -3) \chi _A^2+ \left(\tfrac{21}{2}\eta -6\right) \frac{\chi _A \chi _S}{\delta}+\left(\tfrac{1}{2}\eta -3\right) \chi _S^2  \right] \nonumber\\
&\quad
+ v_\Omega^5 \Big[
\left(-\tfrac{703}{112}\eta^2 +\tfrac{8797}{1008}\eta -\tfrac{81}{16}\right) \frac{\chi _A}{\delta }+\left(\tfrac{613}{1008}\eta^2 +\tfrac{1709}{1008}\eta -\tfrac{81}{16}\right) \chi _S
+ \left(\tfrac{3}{4}-3 \eta \right) \frac{\chi _A^3}{\delta }+ \left(\tfrac{9}{4}-6 \eta \right) \frac{\chi _A \chi _S^2}{\delta }+\left(\tfrac{9}{4}-3 \eta \right) \chi _A^2 \chi _S+\tfrac{3}{4} \chi _S^3 \Big] \nonumber\\
&\quad
+ {v_\Omega^6 \Big[
\left(\tfrac{5}{7}\eta ^2 -\tfrac{9287}{1008}\eta +\tfrac{4163}{252}\right) \chi _A^2
+\left(\tfrac{139}{72} \eta ^2-\tfrac{2633 }{1008}\eta+\tfrac{4163}{252}\right) \chi _S^2
+ \left(\tfrac{9487}{504} \eta ^2 -\tfrac{1636}{21}\eta +\tfrac{4163}{126}\right) \frac{ \chi _A \chi _S}{\delta } \Big]},
\end{align*}

In the limiting case, this amounts to:

\begin{align*}
\lim_{\delta \rightarrow 0} \delta f^{\rm S}_{21} &= -\tfrac{3}{2} v_\Omega \left(\frac{\chi _A}{\delta}\right)
+ v_\Omega^3 \Big[\left(\tfrac{131}{84}\eta +\tfrac{61}{12}\right) \frac{\chi _A}{\delta} \Big]
+ v_\Omega^4 \left[\left(\tfrac{21}{2}\eta -6\right) \frac{\chi _A \chi _S}{\delta}\right] \nonumber\\
&\quad
+ v_\Omega^5 \Big[
\left(-\tfrac{703}{112}\eta^2 +\tfrac{8797}{1008}\eta -\tfrac{81}{16}\right) \frac{\chi _A}{\delta } + \left(\tfrac{3}{4}-3 \eta \right) \frac{\chi _A^3}{\delta }+ \left(\tfrac{9}{4}-6 \eta \right) \frac{\chi _A \chi _S^2}{\delta }\Big] \nonumber\\
&\quad
+ {v_\Omega^6 \Big[\left(\tfrac{9487}{504} \eta ^2 -\tfrac{1636}{21}\eta +\tfrac{4163}{126}\right) \frac{ \chi _A \chi _S}{\delta } \Big]},
\end{align*}


In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
fspin21 = (
    -sp.Rational(3, 2) * v_Omega * (chi_A * deltainv + chi_S)
    + v_Omega**3 * (
        (sp.Rational(131, 84) * eta + sp.Rational(61, 12)) * (chi_A * deltainv)
        + (sp.Rational(79, 84) * eta + sp.Rational(61, 12)) * chi_S
    )
    + v_Omega**4 * (
        (-2 * eta - 3) * chi_A**2
        + (sp.Rational(21, 2) * eta - 6) * (chi_A * chi_S * deltainv)
        + (sp.Rational(1, 2) * eta - 3) * chi_S**2
    )
    + v_Omega**5 * (
        (
            sp.Rational(-703, 112) * eta**2
            + sp.Rational(8797, 1008) * eta
            - sp.Rational(81, 16)
        ) * (chi_A * deltainv)
        + (
            sp.Rational(613, 1008) * eta**2
            + sp.Rational(1709, 1008) * eta
            - sp.Rational(81, 16)
        ) * chi_S
        + (sp.Rational(3, 4) - 3 * eta) * (chi_A**3 * deltainv)
        + (sp.Rational(9, 4) - 6 * eta) * (chi_A * chi_S**2 * deltainv)
        + (sp.Rational(9, 4) - 3 * eta) * (chi_A**2 * chi_S)
        + sp.Rational(3, 4) * chi_S**3
    )
    + v_Omega**6 * (
        (sp.Rational(5, 7) * eta**2 - sp.Rational(9287, 1008) * eta + sp.Rational(4163, 252)) * chi_A**2
        + (sp.Rational(139, 72) * eta**2 - sp.Rational(2633, 1008) * eta + sp.Rational(4163, 252)) * chi_S**2
        + (sp.Rational(9487, 504) * eta**2 - sp.Rational(1636, 21) * eta + sp.Rational(4163, 126)) * (chi_A * chi_S * deltainv)
    )
)

fspin21_limit = (
    -sp.Rational(3, 2) * v_Omega * (chi_A)
    + v_Omega**3 * (
        (sp.Rational(131, 84) * eta + sp.Rational(61, 12)) * (chi_A)
        
    )
    + v_Omega**4 * (
        
        + (sp.Rational(21, 2) * eta - 6) * (chi_A * chi_S)
        
    )
    + v_Omega**5 * (
        (
            sp.Rational(-703, 112) * eta**2
            + sp.Rational(8797, 1008) * eta
            - sp.Rational(81, 16)
        ) * (chi_A)
        + (sp.Rational(3, 4) - 3 * eta) * (chi_A**3)
        + (sp.Rational(9, 4) - 6 * eta) * (chi_A * chi_S**2)
        
    )
    + v_Omega**6 * (
        (sp.Rational(9487, 504) * eta**2 - sp.Rational(1636, 21) * eta + sp.Rational(4163, 126)) * (chi_A * chi_S)
    )
)

The (3,3) mode is given in Equation B3b:

\begin{align*}
\rho_{33}^\text{NS}&= 1+v_\Omega^2 \left(\tfrac{2}{3}\eta-\tfrac{7}{6}\right)  + v_\Omega^4\left(-\tfrac{6719}{3960}-\tfrac{1861}{990}\eta+\tfrac{149 }{330}\eta v_\Omega^2\right) \nonumber \\
&\quad
+v_\Omega^6 \left[\tfrac{3203101567}{227026800}+\left(-\tfrac{129509}{25740}+\tfrac{41 \pi^2}{192}\right) \eta-\tfrac{274621}{154440} \eta^2+\tfrac{12011 }{46332}\eta^3-\tfrac{26}{7} \text{eulerlog}\left(3, v_\Omega\right)\right]  \nonumber \\
&\quad
+v_\Omega^8\left[-\tfrac{57566572157}{8562153600}+\tfrac{13}{3} \text { eulerlog }\left(3, v_\Omega\right)\right] 
+v_\Omega^{10}\left[-\tfrac{903823148417327}{30566888352000}+\tfrac{87347}{13860}\text{eulerlog}\left(3, v_\Omega\right)\right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho33NS = 1 + 
    vomega**2 * (sp.Rational(2, 3) * eta - sp.Rational(7, 6)) + 
    vomega**4 * (-sp.Rational(6719, 3960) - sp.Rational(1861, 990) * eta + sp.Rational(149, 330) * eta * vomega**2) + 
    vomega**6 * (sp.Rational(3203101567, 227026800) + (-sp.Rational(129509, 25740) + sp.Rational(41, 192) * sp.pi**2) * eta -
                sp.Rational(274621, 154440) * eta**2 + sp.Rational(12011, 46332) * eta**3 - sp.Rational(26, 7) * eulerlog3) + 
    vomega**8 * (-sp.Rational(57566572157, 8562153600) + sp.Rational(13, 3) * eulerlog3) + 
    vomega**10 * (-sp.Rational(903823148417327, 30566888352000) + sp.Rational(87347, 13860) * eulerlog3)

The (3,2) mode is given in Equation B6a:

\begin{align*}
\rho_{32} &= 1 + v_\Omega \frac{4 \eta \chi _S}{3 (1-3 \eta )}
+ v_\Omega^2 \left[
\frac{-\tfrac{32}{27}\eta ^2+\tfrac{223}{54}\eta-\tfrac{164}{135}}{1-3 \eta }
{-\frac{16 \eta ^2 \chi _S^2}{9 (1-3 \eta )^2}}
\right] \nonumber\\
&\quad
+ v_\Omega^3 \left[{\left(\tfrac{13 }{9}\eta +\tfrac{2}{9}\right) \frac{\delta \chi _A}{1-3 \eta }
+\left(\tfrac{607}{81}\eta ^3 +\tfrac{503}{81}\eta ^2 -\tfrac{1478}{405}\eta +\tfrac{2}{9}\right)  \frac{\chi _S}{(1-3 \eta )^2} } 
{+\frac{320 \eta ^3 \chi _S^3}{81 (1-3 \eta )^3}}\right]\nonumber\\
&\quad
+ v_\Omega^4 \Bigg[
\frac{\tfrac{77141}{40095} \eta ^4 -\tfrac{508474 }{40095}\eta ^3 -\tfrac{945121 }{320760}\eta ^2 +\tfrac{1610009 \eta }{320760}-\tfrac{180566}{200475}}{(1-3 \eta )^2}
+ {\left(4 \eta ^2-3 \eta +\tfrac{1}{3}\right) \frac{\chi _A^2}{1-3 \eta }
+ \left(-\tfrac{50}{27}\eta ^2-\tfrac{88}{27}\eta+\tfrac{2}{3}\right) \frac{\delta \chi _A \chi _S}{(1-3 \eta )^2} } \nonumber\\
&\qquad
{
+ \left(-\tfrac{2452}{243} \eta ^4 -\tfrac{1997}{243} \eta ^3 +\tfrac{1435}{243}\eta ^2 -\tfrac{43}{27}\eta +\tfrac{1}{3}\right)  \frac{\chi _S^2}{(1-3 \eta )^3}}
\Bigg] \nonumber\\
&\quad
+ {v_\Omega^5 \Bigg[
\left(-\tfrac{1184225 }{96228}\eta ^5 -\tfrac{40204523}{962280} \eta ^4 +\tfrac{101706029 }{962280}\eta ^3 -\tfrac{14103833 }{192456}\eta ^2 +\tfrac{20471053}{962280}\eta -\tfrac{2788}{1215}\right)  \frac{\chi _S}{(1-3 \eta )^3}
+ \left(\tfrac{608}{81} \eta ^3+\tfrac{736}{81} \eta ^2 -\tfrac{16  }{9}\eta\right)  \frac{\delta \chi _A \chi _S^2}{(1-3 \eta )^3}} \nonumber\\
&\qquad
{+\left(\tfrac{889673}{106920}\eta ^3-\tfrac{75737}{5346} \eta ^2+\tfrac{376177 }{35640}\eta -\tfrac{2788}{1215}\right)  \frac{\delta  \chi _A}{(1-3 \eta )^2}
+\left(\tfrac{96176 }{2187}\eta ^5 +\tfrac{43528}{2187} \eta ^4-\tfrac{40232 }{2187}\eta ^3 +\tfrac{376 }{81}\eta ^2 -\tfrac{8 \eta }{9}\right)  \frac{\chi _S^3}{(1-3 \eta )^4}} \nonumber\\
&\qquad
{+\left(-\tfrac{32 }{3}\eta ^3+8 \eta ^2-\tfrac{8}{9}\eta\right)  \frac{\chi _A^2 \chi _S}{(1-3 \eta )^2} \Bigg]}
+v_\Omega^6\left[\tfrac{5849948554}{940355325}-\tfrac{104 \text { eulerlog}(2,v_\Omega)}{63}\right] 
+v_\Omega^8 \left[\tfrac{17056 \text { eulerlog}(2,v_\Omega)}{8505}-\tfrac{10607269449358}{3072140846775}\right] \nonumber \\
&\qquad
+ {v_\Omega^{10}\left[- \tfrac{1312549797426453052}{176264081083715625} +\tfrac{18778864 \text { eulerlog}(2,v_{\Omega} )}{12629925}   \right]},
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho32 = 1 + 
    vomega * (4 * eta * chi_S) / (3 * (1 - 3 * eta)) + 
    vomega**2 * ((sp.Rational(-32, 27) * eta**2 + sp.Rational(223, 54) * eta - sp.Rational(164, 135)) / (1 - 3 * eta) -
                (16 * eta**2 * chi_S**2) / (9 * (1 - 3 * eta)**2)) + 
    vomega**3 * ((sp.Rational(13, 9) * eta + sp.Rational(2, 9)) * (delta * chi_A) / (1 - 3 * eta) +
                (sp.Rational(607, 81) * eta**3 + sp.Rational(503, 81) * eta**2 - sp.Rational(1478, 405) * eta + sp.Rational(2, 9)) * chi_S / (1 - 3 * eta)**2 +
                (320 * eta**3 * chi_S**3) / (81 * (1 - 3 * eta)**3)) + 
    vomega**4 * ((sp.Rational(77141, 40095) * eta**4 - sp.Rational(508474, 40095) * eta**3 - sp.Rational(945121, 320760) * eta**2 +
                sp.Rational(1610009, 320760) * eta - sp.Rational(180566, 200475)) / (1 - 3 * eta)**2 +
                (4 * eta**2 - 3 * eta + sp.Rational(1, 3)) * (chi_A**2) / (1 - 3 * eta) +
                (sp.Rational(-50, 27) * eta**2 - sp.Rational(88, 27) * eta + sp.Rational(2, 3)) * (delta * chi_A * chi_S) / (1 - 3 * eta)**2 +
                (sp.Rational(-2452, 243) * eta**4 - sp.Rational(1997, 243) * eta**3 + sp.Rational(1435, 243) * eta**2 -
                sp.Rational(43, 27) * eta + sp.Rational(1, 3)) * (chi_S**2) / (1 - 3 * eta)**3)) + 
    vomega**5 * ((sp.Rational(-1184225, 96228) * eta**5 - sp.Rational(40204523, 962280) * eta**4 + sp.Rational(101706029, 962280) * eta**3 -
                sp.Rational(14103833, 192456) * eta**2 + sp.Rational(20471053, 962280) * eta - sp.Rational(2788, 1215)) * chi_S / (1 - 3 * eta)**3 +
                (sp.Rational(608, 81) * eta**3 + sp.Rational(736, 81) * eta**2 - sp.Rational(16, 9) * eta) * (delta * chi_A * chi_S**2) / (1 - 3 * eta)**3 +
                (sp.Rational(889673, 106920) * eta**3 - sp.Rational(75737, 5346) * eta**2 + sp.Rational(376177, 35640) * eta -
                sp.Rational(2788, 1215)) * (delta * chi_A) / (1 - 3 * eta)**2 +
                (sp.Rational(96176, 2187) * eta**5 + sp.Rational(43528, 2187) * eta**4 - sp.Rational(40232, 2187) * eta**3 +
                sp.Rational(376, 81) * eta**2 - sp.Rational(8, 9) * eta) * (chi_S**3) / (1 - 3 * eta)**4 +
                (sp.Rational(-32, 3) * eta**3 + 8 * eta**2 - sp.Rational(8, 9) * eta) * (chi_A**2 * chi_S) / (1 - 3 * eta)**2)) + 
    vomega**6 * (sp.Rational(5849948554, 940355325) - sp.Rational(104, 63) * eulerlog2) + 
    vomega**8 * (sp.Rational(17056, 8505) * eulerlog2 - sp.Rational(10607269449358, 3072140846775)) + 
    vomega**10 * (-sp.Rational(1312549797426453052, 176264081083715625) + sp.Rational(18778864, 12629925) * eulerlog2)

The (3,1) mode is given in Equation B8:

\begin{align*}
\rho_{31}^\text{NS} &= 1-v_\Omega^2\left[\frac{2}{9}\eta+\frac{13}{18}\right] 
+ v_\Omega^4  \left[-\frac{829}{1782}\eta^2-\frac{1685}{1782}\eta+\frac{101}{7128}\right] 
+ v_\Omega^6 \left[\frac{11706720301}{6129723600}-\frac{26}{63}\text{eulerlog}(1,v_\Omega^2)\right] \nonumber\\
&\quad
+ v_\Omega^8 \left[\frac{169}{567}\text{eulerlog}(1,v_\Omega^2) +\frac{2606097992581}{4854741091200}\right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho31NS = 1 - 
    vomega**2 * (sp.Rational(2, 9) * eta + sp.Rational(13, 18)) + 
    vomega**4 * (-sp.Rational(829, 1782) * eta**2 - sp.Rational(1685, 1782) * eta + sp.Rational(101, 7128)) + 
    vomega**6 * (sp.Rational(11706720301, 6129723600) - sp.Rational(26, 63) * eulerlog1v2) + 
    vomega**8 * (sp.Rational(169, 567) * eulerlog1v2 + sp.Rational(2606097992581, 4854741091200))

The (4,4) mode is given in Equation B:

\begin{align*}
\rho_{44} &= 1+v_\Omega^2\left[\tfrac{1614-5870 \eta +2625 \eta ^2}{1320 (-1+3 \eta )}\right] + v_\Omega^3 \left[\left(\tfrac{2}{3}-\tfrac{41 \eta }{15}+\tfrac{14 \eta ^2}{5}\right) \tfrac{1}{(-1+3 \eta )}\chi_S 
+\delta \left(\tfrac{2}{3}-\tfrac{13 \eta }{5}\right) \tfrac{1}{(-1+3 \eta )}\chi _A\right]  \nonumber \\
&\quad
+v_\Omega^4\left[-\tfrac{14210377}{8808800 (1-3 \eta )^2}+\tfrac{32485357 \eta
}{4404400 (1-3 \eta )^2}-\tfrac{1401149 \eta ^2}{1415700 (1-3 \eta )^2} 
-\tfrac{801565 \eta ^3}{37752 (1-3 \eta )^2}+\tfrac{3976393 \eta ^4}{1006720 (1-3 \eta )^2}+\tfrac{1}{2}\chi _A^2-2 \eta  \chi _A^2+\delta \chi _A \chi _S+\tfrac{1}{2}\chi _S^2\right]  \nonumber \\
&\quad+ v_\Omega^5\Bigg[\left(-\tfrac{69}{55}+\tfrac{16571 \eta }{1650}-\tfrac{2673 \eta ^2}{100}+\tfrac{8539 \eta ^3}{440}+\tfrac{591 \eta ^4}{44}\right) \tfrac{1}{(1-3 \eta)^2}\chi _S
+\delta
\left(-\tfrac{69}{55}+\tfrac{10679 \eta }{1650}-\tfrac{1933 \eta ^2}{220}+\tfrac{597 \eta ^3}{440}\right) \tfrac{1}{ (1-3 \eta )^2}\chi_A\Bigg]  \nonumber \\
&\quad+v_\Omega^6 \left[\tfrac{16600939332793}{1098809712000}-\tfrac{12568 }{3465}\text{eulerlog}\left(4,v_\Omega\right)\right]  +v_\Omega^8\left[-\tfrac{172066910136202271}{19426955708160000}+\tfrac{845198 }{190575}\text{eulerlog}\left(4,v_\Omega\right)\right] \nonumber \\
&\quad+ v_\Omega^{10}\left[-\tfrac{17154485653213713419357}{568432724020761600000}+\tfrac{22324502267}{3815311500} \text{eulerlog}\left(4,v_\Omega\right)\right]\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho44 = 1 + 
    vomega**2 * ((1614 - 5870 * eta + 2625 * eta**2) / (1320 * (-1 + 3 * eta))) + 
    vomega**3 * ((sp.Rational(2, 3) - sp.Rational(41, 15) * eta + sp.Rational(14, 5) * eta**2) / (-1 + 3 * eta) * chi_S +
                delta * (sp.Rational(2, 3) - sp.Rational(13, 5) * eta) / (-1 + 3 * eta) * chi_A) + 
    vomega**4 * (-sp.Rational(14210377, 8808800) / (1 - 3 * eta)**2 + sp.Rational(32485357, 4404400) * eta / (1 - 3 * eta)**2 -
                sp.Rational(1401149, 1415700) * eta**2 / (1 - 3 * eta)**2 -
                sp.Rational(801565, 37752) * eta**3 / (1 - 3 * eta)**2 +
                sp.Rational(3976393, 1006720) * eta**4 / (1 - 3 * eta)**2 +
                (sp.Rational(1, 2) * chi_A**2 - 2 * eta * chi_A**2 + delta * chi_A * chi_S + sp.Rational(1, 2) * chi_S**2)) + 
    vomega**5 * ((-sp.Rational(69, 55) + sp.Rational(16571, 1650) * eta - sp.Rational(2673, 100) * eta**2 +
                sp.Rational(8539, 440) * eta**3 + sp.Rational(591, 44) * eta**4) / (1 - 3 * eta)**2 * chi_S +
                delta * (-sp.Rational(69, 55) + sp.Rational(10679, 1650) * eta - sp.Rational(1933, 220) * eta**2 +
                sp.Rational(597, 440) * eta**3) / (1 - 3 * eta)**2 * chi_A) + 
    vomega**6 * (sp.Rational(16600939332793, 1098809712000) - sp.Rational(12568, 3465) * eulerlog4) + 
    vomega**8 * (-sp.Rational(172066910136202271, 19426955708160000) + sp.Rational(845198, 190575) * eulerlog4) + 
    vomega**10 * (-sp.Rational(17154485653213713419357, 568432724020761600000) + sp.Rational(22324502267, 3815311500) * eulerlog4)

The (4,3) mode is given in Equation B:

\begin{align*}
\rho_{43}^\text{NS} &= 1 + \frac{v_\Omega^2}{1-2 \eta } \left(-\tfrac{10}{11} \eta ^2+\tfrac{547}{176}\eta-\tfrac{111}{88}\right) -\tfrac{6894273}{7047040} v_\Omega^4
+v_\Omega^6\left[\tfrac{1664224207351}{195343948800}-\tfrac{1571}{770} \text { eulerlog}(3,v_\Omega)\right] \nonumber \\
&\quad
+  v_\Omega^8\left[-\tfrac{2465107182496333}{460490801971200}  + \tfrac{174381}{67760} \text { eulerlog}(3,v_{\Omega} )   \right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS43 = 1 + 
    vomega**2 / (1 - 2 * eta) * (-sp.Rational(10, 11) * eta**2 + sp.Rational(547, 176) * eta - sp.Rational(111, 88)) - 
    sp.Rational(6894273, 7047040) * vomega**4 + 
    vomega**6 * (sp.Rational(1664224207351, 195343948800) - sp.Rational(1571, 770) * eulerlog3) + 
    vomega**8 * (-sp.Rational(2465107182496333, 460490801971200) + sp.Rational(174381, 67760) * eulerlog3)

The (4,2) mode is given in Equation B:

\begin{align*}
\rho_{42}&=1+\frac{285\,\eta^2-3530\,\eta+1146}{1320\,(3\,\eta-1)} v_\Omega^2 - \frac{v_\Omega^3}{15(1-3\eta)} \left[(78\eta^2 - 59\eta + 10)\chi_S +(10-21\eta)\delta\,\chi_A \right] \nonumber \\
&\quad+\frac{-379526805\,\eta^4-3047981160\,\eta^3+1204388696\,\eta^2+295834536\,\eta-114859044}{317116800\,(1-3\,\eta)^2} v_\Omega^4 \nonumber\\
&\quad+ \left[\frac{848238724511}{219761942400}-\frac{3142}{3465}\text{eulerlog}(2,v_\Omega^2)\right] v_\Omega^6\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho42 = 1 + 
    ((285 * eta**2 - 3530 * eta + 1146) / (1320 * (3 * eta - 1))) * vomega**2 - 
    (vomega**3 / (15 * (1 - 3 * eta))) * ((78 * eta**2 - 59 * eta + 10) * chi_S + (10 - 21 * eta) * delta * chi_A) - 
    ((-379526805 * eta**4 - 3047981160 * eta**3 + 1204388696 * eta**2 + 295834536 * eta - 114859044) /
    (317116800 * (1 - 3 * eta)**2)) * vomega**4 + 
    ((848238724511 / 219761942400) - (3142 / 3465) * eulerlog2v2) * vomega**6

The (4,1) mode is given in Equation B:

\begin{align*}
\rho_{41}^\text{NS} &= 1 +\frac{288\,\eta^2-1385\,\eta+602}{528\,(2\,\eta-1)}v_\Omega^2
-\frac{7775491}{21141120} v_\Omega^4 + \left[\frac{1227423222031}{1758095539200}-\frac{1571}{6930}\text{eulerlog}(1,v_\Omega^2)\right]\, v_\Omega^6 \,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS41 = 1 + 
    ((288 * eta**2 - 1385 * eta + 602) / (528 * (2 * eta - 1))) * vomega**2 - 
    (sp.Rational(7775491, 21141120)) * vomega**4 + 
    ((sp.Rational(1227423222031, 1758095539200) - sp.Rational(1571, 6930) * eulerlog1v2) * vomega**6)

The (5,5) mode is given in Equation B:

\begin{align*}
\rho_{55}^{\mathrm{NS}} =& 1+v_\Omega^2\left[\tfrac{487}{390 (-1+2 \eta )}-\tfrac{649 \eta }{195 (-1+2 \eta )}+\tfrac{256 \eta ^2}{195 (-1+2 \eta )}\right] -\tfrac{3353747}{2129400} v_\Omega^4 \nonumber \\
   &+v_\Omega^6\left[\tfrac{190606537999247}{11957879934000}-\tfrac{1546}{429}\text{eulerlog}\left(5,v_\Omega\right) \right] +v_\Omega^8 \left[-\tfrac{1213641959949291437}{118143853747920000}+\tfrac{376451 }{83655}\text{eulerlog}\left(5,v_\Omega\right)\right] \nonumber \\
   &+v_\Omega^{10}\left[-\tfrac{150082616449726042201261}{4837990810977324000000}+\tfrac{2592446431}{456756300}\text{eulerlog}\left(5,v_\Omega\right)\right]\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS55 = 1 + \
    vomega**2 * (sp.Rational(487, 390) / (-1 + 2 * eta) - sp.Rational(649, 195) * eta / (-1 + 2 * eta) +
                 sp.Rational(256, 195) * eta**2 / (-1 + 2 * eta)) - \
    sp.Rational(3353747, 2129400) * vomega**4 + \
    (sp.Rational(190606537999247, 11957879934000) - sp.Rational(1546, 429) * eulerlog5) * vomega**6 + \
    (-sp.Rational(1213641959949291437, 118143853747920000) + sp.Rational(376451, 83655) * eulerlog5) * vomega**8 + \
    (-sp.Rational(150082616449726042201261, 4837990810977324000000) + sp.Rational(2592446431, 456756300) * eulerlog5) * vomega**10


<a id='rho'></a>

# Step : $\rho_{lm}$ \[Back to [top](#toc)\]
$$\label{rho}$$

$\rho_{lm}$ is defined in Appendix B of [PB2023](https://arxiv.org/pdf/2303.18039.pdf).

We begin with the (2,2) mode given in Equation B1a:

\begin{align*}
\rho_{22} &= 1
+ v_\Omega^2 \left(\tfrac{55}{84}\eta-\tfrac{43}{42}\right)
+ v_\Omega^3 \left[\left(\tfrac{2}{3}\eta-\tfrac{2}{3}\right) \chi _S-\tfrac{2}{3}\delta\chi _A\right]
+ v_\Omega^4 \Big[
\tfrac{19583}{42336} \eta^2-\tfrac{33025}{21168}\eta-\tfrac{20555}{10584}
+ \left(\tfrac{1}{2}-2 \eta \right) \chi _A^2+\delta  \chi _A \chi _S+\tfrac{1}{2}\chi _S^2\Big] \nonumber\\
&\quad
+ v_\Omega^5 \left[\delta  \left(-\tfrac{19}{42}\eta-\tfrac{34}{21}\right) \chi _A+\left(\tfrac{209}{126}\eta^2+\tfrac{49}{18}\eta-\tfrac{34}{21}\right) \chi _S\right] \nonumber \\
&\quad
+ v_\Omega^6 \Big[
\tfrac{10620745 \eta^3}{39118464}-\tfrac{6292061 \eta^2}{3259872}+\tfrac{41 \pi^2 \eta}{192}-\tfrac{48993925 \eta}{9779616}-\tfrac{428}{105}\text{eulerlog}(2,v_\Omega)+\tfrac{1556919113}{122245200} \nonumber \\
&\qquad 
 + \delta  \left(\tfrac{89}{126}-\tfrac{781}{252}\eta\right) \chi _A \chi _S
+\left(-\tfrac{27}{14}\eta^2-\tfrac{457}{504}\eta+\tfrac{89}{252}\right) \chi _A^2
+\left(\tfrac{10}{9}\eta^2 -\tfrac{1817}{504}\eta +\tfrac{89}{252}\right) \chi _S^2 \Big] \nonumber\\
&\quad
+ v_\Omega^7 \Big[
\delta  \left(\tfrac{97865}{63504}\eta^2 +\tfrac{50140}{3969}\eta +\tfrac{18733}{15876}\right) \chi _A
+\left(\tfrac{50803}{63504}\eta^3 -\tfrac{245717}{63504}\eta^2 +\tfrac{74749 }{5292}\eta +\tfrac{18733}{15876}\right) \chi _S
{+ \delta \chi _A^3 \left(\tfrac{1}{3}-\tfrac{4}{3}\eta\right)+\delta  (2 \eta +1) \chi _A \chi _S^2} \nonumber\\
&\qquad
{+ \left(-4 \eta ^2-3 \eta +1\right) \chi _A^2 \chi _S+\left(\eta +\tfrac{1}{3}\right) \chi _S^3}\Big] 
+ v_\Omega^8 \left[\tfrac{9202}{2205}\text{eulerlog}(2,v_\Omega)-\tfrac{387216563023}{160190110080}\right]\nonumber \\
&\quad 
+ v_\Omega^{10} \left[\tfrac{439877}{55566}\text {eulerlog}(2,v_\Omega)-\tfrac{16094530514677}{533967033600}\right], \\
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho22 = 1 + 
    vomega**2 * (sp.Rational(55, 84) * eta - sp.Rational(43, 42)) + 
    vomega**3 * ((sp.Rational(2, 3) * eta - sp.Rational(2, 3)) * chi_S - sp.Rational(2, 3) * delta * chi_A) +
    vomega**4 * (sp.Rational(19583, 42336) * eta**2 - sp.Rational(33025, 21168) * eta - sp.Rational(20555, 10584) +
                (sp.Rational(1, 2) - 2 * eta) * chi_A**2 + delta * chi_A * chi_S + sp.Rational(1, 2) * chi_S**2) + 
    vomega**5 * (delta * (sp.Rational(-19, 42) * eta - sp.Rational(34, 21)) * chi_A +
                (sp.Rational(209, 126) * eta**2 + sp.Rational(49, 18) * eta - sp.Rational(34, 21)) * chi_S) + 
    vomega**6 * (sp.Rational(10620745, 39118464) * eta**3 - sp.Rational(6292061, 3259872) * eta**2 +
                sp.Rational(41, 192) * sp.pi**2 * eta - sp.Rational(48993925, 9779616) * eta -
                sp.Rational(428, 105) * eulerlog2 + sp.Rational(1556919113, 122245200) +
                delta * (sp.Rational(89, 126) - sp.Rational(781, 252) * eta) * chi_A * chi_S +
                (sp.Rational(-27, 14) * eta**2 - sp.Rational(457, 504) * eta + sp.Rational(89, 252)) * chi_A**2 +
                (sp.Rational(10, 9) * eta**2 - sp.Rational(1817, 504) * eta + sp.Rational(89, 252)) * chi_S**2) + 
    vomega**7 * (delta * (sp.Rational(97865, 63504) * eta**2 + sp.Rational(50140, 3969) * eta + sp.Rational(18733, 15876)) * chi_A +
                (sp.Rational(50803, 63504) * eta**3 - sp.Rational(245717, 63504) * eta**2 +
                sp.Rational(74749, 5292) * eta + sp.Rational(18733, 15876)) * chi_S +
                delta * chi_A**3 * (sp.Rational(1, 3) - sp.Rational(4, 3) * eta) +
                delta * (2 * eta + 1) * chi_A * chi_S**2 +
                (sp.Rational(-4, 1) * eta**2 - sp.Rational(3, 1) * eta + sp.Rational(1, 1)) * chi_A**2 * chi_S +
                (eta + sp.Rational(1, 3)) * chi_S**3) + 
    vomega**8 * (sp.Rational(9202, 2205) * eulerlog2 - sp.Rational(387216563023, 160190110080)) + 
    vomega**10 * (sp.Rational(439877, 55566) * eulerlog2 - sp.Rational(16094530514677, 533967033600))

The (2,1) mode is given in Equation B2a:

\begin{align*}
\rho_{21}^\text{NS} &=1
+v_\Omega^2 \left(\tfrac{23}{84}\eta -\tfrac{59}{56}\right) 
+v_\Omega^4 \left(\tfrac{617}{4704}\eta^2 -\tfrac{10993}{14112}\eta -\tfrac{47009}{56448}\right)  +v_\Omega^6 \left[\tfrac{7613184941}{2607897600}-\tfrac{107}{105} \text { eulerlog }\left(1, v_\Omega\right)\right]  \nonumber \\
&\quad+ v_\Omega^8\left[-\tfrac{1168617463883}{911303737344}+\tfrac{6313}{5880}\text { eulerlog }\left(1, v_\Omega\right)\right]
+v_\Omega^{10}\left[-\tfrac{63735873771463}{16569158860800} + \tfrac{5029963}{5927040} \text{ eulerlog }\left(1, v_\Omega\right)\right] ,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho21NS = 1 + 
    vomega**2 * (sp.Rational(23, 84) * eta - sp.Rational(59, 56)) + 
    vomega**4 * (sp.Rational(617, 4704) * eta**2 - sp.Rational(10993, 14112) * eta - sp.Rational(47009, 56448)) + 
    vomega**6 * (sp.Rational(7613184941, 2607897600) - sp.Rational(107, 105) * eulerlog1) + 
    vomega**8 * (-sp.Rational(1168617463883, 911303737344) + sp.Rational(6313, 5880) * eulerlog1) + 
    vomega**10 * (-sp.Rational(63735873771463, 16569158860800) + sp.Rational(5029963, 5927040) * eulerlog1)

The (3,3) mode is given in Equation B3a:

\begin{align*}
\rho_{33}^\text{NS}&= 1+v_\Omega^2 \left(\tfrac{2}{3}\eta-\tfrac{7}{6}\right)  + v_\Omega^4\left(-\tfrac{6719}{3960}-\tfrac{1861}{990}\eta+\tfrac{149 }{330}\eta v_\Omega^2\right) \nonumber \\
&\quad
+v_\Omega^6 \left[\tfrac{3203101567}{227026800}+\left(-\tfrac{129509}{25740}+\tfrac{41 \pi^2}{192}\right) \eta-\tfrac{274621}{154440} \eta^2+\tfrac{12011 }{46332}\eta^3-\tfrac{26}{7} \text{eulerlog}\left(3, v_\Omega\right)\right]  \nonumber \\
&\quad
+v_\Omega^8\left[-\tfrac{57566572157}{8562153600}+\tfrac{13}{3} \text { eulerlog }\left(3, v_\Omega\right)\right] 
+v_\Omega^{10}\left[-\tfrac{903823148417327}{30566888352000}+\tfrac{87347}{13860}\text{eulerlog}\left(3, v_\Omega\right)\right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho33NS = 1 + 
    vomega**2 * (sp.Rational(2, 3) * eta - sp.Rational(7, 6)) + 
    vomega**4 * (-sp.Rational(6719, 3960) - sp.Rational(1861, 990) * eta + sp.Rational(149, 330) * eta * vomega**2) + 
    vomega**6 * (sp.Rational(3203101567, 227026800) + (-sp.Rational(129509, 25740) + sp.Rational(41, 192) * sp.pi**2) * eta -
                sp.Rational(274621, 154440) * eta**2 + sp.Rational(12011, 46332) * eta**3 - sp.Rational(26, 7) * eulerlog3) + 
    vomega**8 * (-sp.Rational(57566572157, 8562153600) + sp.Rational(13, 3) * eulerlog3) + 
    vomega**10 * (-sp.Rational(903823148417327, 30566888352000) + sp.Rational(87347, 13860) * eulerlog3)

The (3,2) mode is given in Equation B6a:

\begin{align*}
\rho_{32} &= 1 + v_\Omega \frac{4 \eta \chi _S}{3 (1-3 \eta )}
+ v_\Omega^2 \left[
\frac{-\tfrac{32}{27}\eta ^2+\tfrac{223}{54}\eta-\tfrac{164}{135}}{1-3 \eta }
{-\frac{16 \eta ^2 \chi _S^2}{9 (1-3 \eta )^2}}
\right] \nonumber\\
&\quad
+ v_\Omega^3 \left[{\left(\tfrac{13 }{9}\eta +\tfrac{2}{9}\right) \frac{\delta \chi _A}{1-3 \eta }
+\left(\tfrac{607}{81}\eta ^3 +\tfrac{503}{81}\eta ^2 -\tfrac{1478}{405}\eta +\tfrac{2}{9}\right)  \frac{\chi _S}{(1-3 \eta )^2} } 
{+\frac{320 \eta ^3 \chi _S^3}{81 (1-3 \eta )^3}}\right]\nonumber\\
&\quad
+ v_\Omega^4 \Bigg[
\frac{\tfrac{77141}{40095} \eta ^4 -\tfrac{508474 }{40095}\eta ^3 -\tfrac{945121 }{320760}\eta ^2 +\tfrac{1610009 \eta }{320760}-\tfrac{180566}{200475}}{(1-3 \eta )^2}
+ {\left(4 \eta ^2-3 \eta +\tfrac{1}{3}\right) \frac{\chi _A^2}{1-3 \eta }
+ \left(-\tfrac{50}{27}\eta ^2-\tfrac{88}{27}\eta+\tfrac{2}{3}\right) \frac{\delta \chi _A \chi _S}{(1-3 \eta )^2} } \nonumber\\
&\qquad
{
+ \left(-\tfrac{2452}{243} \eta ^4 -\tfrac{1997}{243} \eta ^3 +\tfrac{1435}{243}\eta ^2 -\tfrac{43}{27}\eta +\tfrac{1}{3}\right)  \frac{\chi _S^2}{(1-3 \eta )^3}}
\Bigg] \nonumber\\
&\quad
+ {v_\Omega^5 \Bigg[
\left(-\tfrac{1184225 }{96228}\eta ^5 -\tfrac{40204523}{962280} \eta ^4 +\tfrac{101706029 }{962280}\eta ^3 -\tfrac{14103833 }{192456}\eta ^2 +\tfrac{20471053}{962280}\eta -\tfrac{2788}{1215}\right)  \frac{\chi _S}{(1-3 \eta )^3}
+ \left(\tfrac{608}{81} \eta ^3+\tfrac{736}{81} \eta ^2 -\tfrac{16  }{9}\eta\right)  \frac{\delta \chi _A \chi _S^2}{(1-3 \eta )^3}} \nonumber\\
&\qquad
{+\left(\tfrac{889673}{106920}\eta ^3-\tfrac{75737}{5346} \eta ^2+\tfrac{376177 }{35640}\eta -\tfrac{2788}{1215}\right)  \frac{\delta  \chi _A}{(1-3 \eta )^2}
+\left(\tfrac{96176 }{2187}\eta ^5 +\tfrac{43528}{2187} \eta ^4-\tfrac{40232 }{2187}\eta ^3 +\tfrac{376 }{81}\eta ^2 -\tfrac{8 \eta }{9}\right)  \frac{\chi _S^3}{(1-3 \eta )^4}} \nonumber\\
&\qquad
{+\left(-\tfrac{32 }{3}\eta ^3+8 \eta ^2-\tfrac{8}{9}\eta\right)  \frac{\chi _A^2 \chi _S}{(1-3 \eta )^2} \Bigg]}
+v_\Omega^6\left[\tfrac{5849948554}{940355325}-\tfrac{104 \text { eulerlog}(2,v_\Omega)}{63}\right] 
+v_\Omega^8 \left[\tfrac{17056 \text { eulerlog}(2,v_\Omega)}{8505}-\tfrac{10607269449358}{3072140846775}\right] \nonumber \\
&\qquad
+ {v_\Omega^{10}\left[- \tfrac{1312549797426453052}{176264081083715625} +\tfrac{18778864 \text { eulerlog}(2,v_{\Omega} )}{12629925}   \right]},
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho32 = 1 + 
    vomega * (4 * eta * chi_S) / (3 * (1 - 3 * eta)) + 
    vomega**2 * ((sp.Rational(-32, 27) * eta**2 + sp.Rational(223, 54) * eta - sp.Rational(164, 135)) / (1 - 3 * eta) -
                (16 * eta**2 * chi_S**2) / (9 * (1 - 3 * eta)**2)) + 
    vomega**3 * ((sp.Rational(13, 9) * eta + sp.Rational(2, 9)) * (delta * chi_A) / (1 - 3 * eta) +
                (sp.Rational(607, 81) * eta**3 + sp.Rational(503, 81) * eta**2 - sp.Rational(1478, 405) * eta + sp.Rational(2, 9)) * chi_S / (1 - 3 * eta)**2 +
                (320 * eta**3 * chi_S**3) / (81 * (1 - 3 * eta)**3)) + 
    vomega**4 * ((sp.Rational(77141, 40095) * eta**4 - sp.Rational(508474, 40095) * eta**3 - sp.Rational(945121, 320760) * eta**2 +
                sp.Rational(1610009, 320760) * eta - sp.Rational(180566, 200475)) / (1 - 3 * eta)**2 +
                (4 * eta**2 - 3 * eta + sp.Rational(1, 3)) * (chi_A**2) / (1 - 3 * eta) +
                (sp.Rational(-50, 27) * eta**2 - sp.Rational(88, 27) * eta + sp.Rational(2, 3)) * (delta * chi_A * chi_S) / (1 - 3 * eta)**2 +
                (sp.Rational(-2452, 243) * eta**4 - sp.Rational(1997, 243) * eta**3 + sp.Rational(1435, 243) * eta**2 -
                sp.Rational(43, 27) * eta + sp.Rational(1, 3)) * (chi_S**2) / (1 - 3 * eta)**3)) + 
    vomega**5 * ((sp.Rational(-1184225, 96228) * eta**5 - sp.Rational(40204523, 962280) * eta**4 + sp.Rational(101706029, 962280) * eta**3 -
                sp.Rational(14103833, 192456) * eta**2 + sp.Rational(20471053, 962280) * eta - sp.Rational(2788, 1215)) * chi_S / (1 - 3 * eta)**3 +
                (sp.Rational(608, 81) * eta**3 + sp.Rational(736, 81) * eta**2 - sp.Rational(16, 9) * eta) * (delta * chi_A * chi_S**2) / (1 - 3 * eta)**3 +
                (sp.Rational(889673, 106920) * eta**3 - sp.Rational(75737, 5346) * eta**2 + sp.Rational(376177, 35640) * eta -
                sp.Rational(2788, 1215)) * (delta * chi_A) / (1 - 3 * eta)**2 +
                (sp.Rational(96176, 2187) * eta**5 + sp.Rational(43528, 2187) * eta**4 - sp.Rational(40232, 2187) * eta**3 +
                sp.Rational(376, 81) * eta**2 - sp.Rational(8, 9) * eta) * (chi_S**3) / (1 - 3 * eta)**4 +
                (sp.Rational(-32, 3) * eta**3 + 8 * eta**2 - sp.Rational(8, 9) * eta) * (chi_A**2 * chi_S) / (1 - 3 * eta)**2)) + 
    vomega**6 * (sp.Rational(5849948554, 940355325) - sp.Rational(104, 63) * eulerlog2) + 
    vomega**8 * (sp.Rational(17056, 8505) * eulerlog2 - sp.Rational(10607269449358, 3072140846775)) + 
    vomega**10 * (-sp.Rational(1312549797426453052, 176264081083715625) + sp.Rational(18778864, 12629925) * eulerlog2)

The (3,1) mode is given in Equation B8:

\begin{align*}
\rho_{31}^\text{NS} &= 1-v_\Omega^2\left[\frac{2}{9}\eta+\frac{13}{18}\right] 
+ v_\Omega^4  \left[-\frac{829}{1782}\eta^2-\frac{1685}{1782}\eta+\frac{101}{7128}\right] 
+ v_\Omega^6 \left[\frac{11706720301}{6129723600}-\frac{26}{63}\text{eulerlog}(1,v_\Omega^2)\right] \nonumber\\
&\quad
+ v_\Omega^8 \left[\frac{169}{567}\text{eulerlog}(1,v_\Omega^2) +\frac{2606097992581}{4854741091200}\right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho31NS = 1 - 
    vomega**2 * (sp.Rational(2, 9) * eta + sp.Rational(13, 18)) + 
    vomega**4 * (-sp.Rational(829, 1782) * eta**2 - sp.Rational(1685, 1782) * eta + sp.Rational(101, 7128)) + 
    vomega**6 * (sp.Rational(11706720301, 6129723600) - sp.Rational(26, 63) * eulerlog1v2) + 
    vomega**8 * (sp.Rational(169, 567) * eulerlog1v2 + sp.Rational(2606097992581, 4854741091200))

The (4,4) mode is given in Equation B:

\begin{align*}
\rho_{44} &= 1+v_\Omega^2\left[\tfrac{1614-5870 \eta +2625 \eta ^2}{1320 (-1+3 \eta )}\right] + v_\Omega^3 \left[\left(\tfrac{2}{3}-\tfrac{41 \eta }{15}+\tfrac{14 \eta ^2}{5}\right) \tfrac{1}{(-1+3 \eta )}\chi_S 
+\delta \left(\tfrac{2}{3}-\tfrac{13 \eta }{5}\right) \tfrac{1}{(-1+3 \eta )}\chi _A\right]  \nonumber \\
&\quad
+v_\Omega^4\left[-\tfrac{14210377}{8808800 (1-3 \eta )^2}+\tfrac{32485357 \eta
}{4404400 (1-3 \eta )^2}-\tfrac{1401149 \eta ^2}{1415700 (1-3 \eta )^2} 
-\tfrac{801565 \eta ^3}{37752 (1-3 \eta )^2}+\tfrac{3976393 \eta ^4}{1006720 (1-3 \eta )^2}+\tfrac{1}{2}\chi _A^2-2 \eta  \chi _A^2+\delta \chi _A \chi _S+\tfrac{1}{2}\chi _S^2\right]  \nonumber \\
&\quad+ v_\Omega^5\Bigg[\left(-\tfrac{69}{55}+\tfrac{16571 \eta }{1650}-\tfrac{2673 \eta ^2}{100}+\tfrac{8539 \eta ^3}{440}+\tfrac{591 \eta ^4}{44}\right) \tfrac{1}{(1-3 \eta)^2}\chi _S
+\delta
\left(-\tfrac{69}{55}+\tfrac{10679 \eta }{1650}-\tfrac{1933 \eta ^2}{220}+\tfrac{597 \eta ^3}{440}\right) \tfrac{1}{ (1-3 \eta )^2}\chi_A\Bigg]  \nonumber \\
&\quad+v_\Omega^6 \left[\tfrac{16600939332793}{1098809712000}-\tfrac{12568 }{3465}\text{eulerlog}\left(4,v_\Omega\right)\right]  +v_\Omega^8\left[-\tfrac{172066910136202271}{19426955708160000}+\tfrac{845198 }{190575}\text{eulerlog}\left(4,v_\Omega\right)\right] \nonumber \\
&\quad+ v_\Omega^{10}\left[-\tfrac{17154485653213713419357}{568432724020761600000}+\tfrac{22324502267}{3815311500} \text{eulerlog}\left(4,v_\Omega\right)\right]\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho44 = 1 + 
    vomega**2 * ((1614 - 5870 * eta + 2625 * eta**2) / (1320 * (-1 + 3 * eta))) + 
    vomega**3 * ((sp.Rational(2, 3) - sp.Rational(41, 15) * eta + sp.Rational(14, 5) * eta**2) / (-1 + 3 * eta) * chi_S +
                delta * (sp.Rational(2, 3) - sp.Rational(13, 5) * eta) / (-1 + 3 * eta) * chi_A) + 
    vomega**4 * (-sp.Rational(14210377, 8808800) / (1 - 3 * eta)**2 + sp.Rational(32485357, 4404400) * eta / (1 - 3 * eta)**2 -
                sp.Rational(1401149, 1415700) * eta**2 / (1 - 3 * eta)**2 -
                sp.Rational(801565, 37752) * eta**3 / (1 - 3 * eta)**2 +
                sp.Rational(3976393, 1006720) * eta**4 / (1 - 3 * eta)**2 +
                (sp.Rational(1, 2) * chi_A**2 - 2 * eta * chi_A**2 + delta * chi_A * chi_S + sp.Rational(1, 2) * chi_S**2)) + 
    vomega**5 * ((-sp.Rational(69, 55) + sp.Rational(16571, 1650) * eta - sp.Rational(2673, 100) * eta**2 +
                sp.Rational(8539, 440) * eta**3 + sp.Rational(591, 44) * eta**4) / (1 - 3 * eta)**2 * chi_S +
                delta * (-sp.Rational(69, 55) + sp.Rational(10679, 1650) * eta - sp.Rational(1933, 220) * eta**2 +
                sp.Rational(597, 440) * eta**3) / (1 - 3 * eta)**2 * chi_A) + 
    vomega**6 * (sp.Rational(16600939332793, 1098809712000) - sp.Rational(12568, 3465) * eulerlog4) + 
    vomega**8 * (-sp.Rational(172066910136202271, 19426955708160000) + sp.Rational(845198, 190575) * eulerlog4) + 
    vomega**10 * (-sp.Rational(17154485653213713419357, 568432724020761600000) + sp.Rational(22324502267, 3815311500) * eulerlog4)

The (4,3) mode is given in Equation B:

\begin{align*}
\rho_{43}^\text{NS} &= 1 + \frac{v_\Omega^2}{1-2 \eta } \left(-\tfrac{10}{11} \eta ^2+\tfrac{547}{176}\eta-\tfrac{111}{88}\right) -\tfrac{6894273}{7047040} v_\Omega^4
+v_\Omega^6\left[\tfrac{1664224207351}{195343948800}-\tfrac{1571}{770} \text { eulerlog}(3,v_\Omega)\right] \nonumber \\
&\quad
+  v_\Omega^8\left[-\tfrac{2465107182496333}{460490801971200}  + \tfrac{174381}{67760} \text { eulerlog}(3,v_{\Omega} )   \right],
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS43 = 1 + 
    vomega**2 / (1 - 2 * eta) * (-sp.Rational(10, 11) * eta**2 + sp.Rational(547, 176) * eta - sp.Rational(111, 88)) - 
    sp.Rational(6894273, 7047040) * vomega**4 + 
    vomega**6 * (sp.Rational(1664224207351, 195343948800) - sp.Rational(1571, 770) * eulerlog3) + 
    vomega**8 * (-sp.Rational(2465107182496333, 460490801971200) + sp.Rational(174381, 67760) * eulerlog3)

The (4,2) mode is given in Equation B:

\begin{align*}
\rho_{42}&=1+\frac{285\,\eta^2-3530\,\eta+1146}{1320\,(3\,\eta-1)} v_\Omega^2 - \frac{v_\Omega^3}{15(1-3\eta)} \left[(78\eta^2 - 59\eta + 10)\chi_S +(10-21\eta)\delta\,\chi_A \right] \nonumber \\
&\quad+\frac{-379526805\,\eta^4-3047981160\,\eta^3+1204388696\,\eta^2+295834536\,\eta-114859044}{317116800\,(1-3\,\eta)^2} v_\Omega^4 \nonumber\\
&\quad+ \left[\frac{848238724511}{219761942400}-\frac{3142}{3465}\text{eulerlog}(2,v_\Omega^2)\right] v_\Omega^6\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rho42 = 1 + 
    ((285 * eta**2 - 3530 * eta + 1146) / (1320 * (3 * eta - 1))) * vomega**2 - 
    (vomega**3 / (15 * (1 - 3 * eta))) * ((78 * eta**2 - 59 * eta + 10) * chi_S + (10 - 21 * eta) * delta * chi_A) - 
    ((-379526805 * eta**4 - 3047981160 * eta**3 + 1204388696 * eta**2 + 295834536 * eta - 114859044) /
    (317116800 * (1 - 3 * eta)**2)) * vomega**4 + 
    ((848238724511 / 219761942400) - (3142 / 3465) * eulerlog2v2) * vomega**6

The (4,1) mode is given in Equation B:

\begin{align*}
\rho_{41}^\text{NS} &= 1 +\frac{288\,\eta^2-1385\,\eta+602}{528\,(2\,\eta-1)}v_\Omega^2
-\frac{7775491}{21141120} v_\Omega^4 + \left[\frac{1227423222031}{1758095539200}-\frac{1571}{6930}\text{eulerlog}(1,v_\Omega^2)\right]\, v_\Omega^6 \,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS41 = 1 + 
    ((288 * eta**2 - 1385 * eta + 602) / (528 * (2 * eta - 1))) * vomega**2 - 
    (sp.Rational(7775491, 21141120)) * vomega**4 + 
    ((sp.Rational(1227423222031, 1758095539200) - sp.Rational(1571, 6930) * eulerlog1v2) * vomega**6)

The (5,5) mode is given in Equation B:

\begin{align*}
\rho_{55}^{\mathrm{NS}} =& 1+v_\Omega^2\left[\tfrac{487}{390 (-1+2 \eta )}-\tfrac{649 \eta }{195 (-1+2 \eta )}+\tfrac{256 \eta ^2}{195 (-1+2 \eta )}\right] -\tfrac{3353747}{2129400} v_\Omega^4 \nonumber \\
   &+v_\Omega^6\left[\tfrac{190606537999247}{11957879934000}-\tfrac{1546}{429}\text{eulerlog}\left(5,v_\Omega\right) \right] +v_\Omega^8 \left[-\tfrac{1213641959949291437}{118143853747920000}+\tfrac{376451 }{83655}\text{eulerlog}\left(5,v_\Omega\right)\right] \nonumber \\
   &+v_\Omega^{10}\left[-\tfrac{150082616449726042201261}{4837990810977324000000}+\tfrac{2592446431}{456756300}\text{eulerlog}\left(5,v_\Omega\right)\right]\,,
\end{align*}

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Waveform-hlm_on_top.txt
rhoNS55 = 1 + \
    vomega**2 * (sp.Rational(487, 390) / (-1 + 2 * eta) - sp.Rational(649, 195) * eta / (-1 + 2 * eta) +
                 sp.Rational(256, 195) * eta**2 / (-1 + 2 * eta)) - \
    sp.Rational(3353747, 2129400) * vomega**4 + \
    (sp.Rational(190606537999247, 11957879934000) - sp.Rational(1546, 429) * eulerlog5) * vomega**6 + \
    (-sp.Rational(1213641959949291437, 118143853747920000) + sp.Rational(376451, 83655) * eulerlog5) * vomega**8 + \
    (-sp.Rational(150082616449726042201261, 4837990810977324000000) + sp.Rational(2592446431, 456756300) * eulerlog5) * vomega**10


\delta_{54}&= \frac{8}{15} \Omega H_\text{EOB},\\
\delta_{53}&= \frac{31}{70} \Omega H_\text{EOB}, \\
\delta_{52}&= \frac{4}{15} \Omega H_\text{EOB}, \\
\delta_{51}&= \frac{31}{210} \Omega H_\text{EOB},

\delta_{66}&= \frac{43}{70} \Omega H_\text{EOB}, \\
\delta_{65}&= \frac{10}{21} \Omega H_\text{EOB}, \\
\delta_{64}&= \frac{43}{105} \Omega H_\text{EOB}, \\
\delta_{63}&= \frac{2}{7} \Omega H_\text{EOB},\\
\delta_{62}&= \frac{43}{210} \Omega H_\text{EOB},\\
\delta_{61}&= \frac{2}{21} \Omega H_\text{EOB},\\

\delta_{77}&= \frac{19}{36} \Omega H_\text{EOB},\\
\delta_{76}&= 0,\\
\delta_{75}&= \frac{95}{252} \Omega H_\text{EOB},\\
\delta_{74}&= 0
\delta_{73}&= \frac{19}{84} \Omega H_\text{EOB},\\
\delta_{71}&= \frac{19}{252} \Omega H_\text{EOB},\\